This notebook contains model analysis code for the model comparison results reported in the conference submission "Learning planning strategies without feedback" - https://osf.io/fyx4g/

In [ ]:
import csv
import pickle
import json
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from mouselab.mouselab import MouselabEnv
import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy
import math
import sklearn.cluster
from math import floor, ceil

# Reading Data from Files

In [ ]:
fileprefix = "final" # pilot_#, final
datafolder = '../results/anonymized_data/'

all_part_files = [filename for filename in os.listdir(datafolder) if filename.startswith(fileprefix) and 'results' not in filename]
all_part_paths = [datafolder + filename for filename in all_part_files]

# Creating Trial Dataframe

In [ ]:
# Each participant has their own file, generated from the Heroku postgres output in the 
# preprocessing notebook

part_stats_dicts = []
scarcity_level = 0.25
for file in all_part_paths:
    with open(file, 'r') as f:
        participant = json.load(f)
    try:
        beginhit = datetime.datetime.strptime(participant['Beginhit'], '%Y-%m-%d %H:%M:%S.%f')
    except TypeError:
        beginhit = None
    try:
        endhit = datetime.datetime.strptime(participant['Endhit'], '%Y-%m-%d %H:%M:%S.%f')
    except TypeError:
        endhit = None
    try:    
        beginexp = datetime.datetime.strptime(participant['Beginexp'], '%Y-%m-%d %H:%M:%S.%f')
    except TypeError:
        beginexp = None
    
    try:
        hitLength = (endhit-beginhit).seconds
    except TypeError:
        hitLength = None
    
    try:
        expLength = (endhit-beginexp).seconds
    except TypeError:
        expLength = None
        
    numPracTrials = 0
    trialLengths = []
    trialScores = []
    expectedScores = []
    numClicks = []
    avgClickLevel = []
    rewardsWithheld = []
    demographicsAvailable = False
    effort = "-1"
    age = None
    gender = None
    colorblind = None
    
    print(participant['workerId'])
    try:
        firstTrialStamp = participant['data'][0]['dateTime']
    except:
        print("No data")
        continue
    lastTrialStamp = participant['data'][-1]['dateTime']
    if hitLength is None:
        hitLength = (lastTrialStamp - firstTrialStamp) / 1000
    if expLength is None:
        expLength = (lastTrialStamp - firstTrialStamp) / 1000
        
    # Dictionary to check whether stroop tasks have been completed
    stroop_timestamps = {
        "1" : {
            "start" : 0,
            "end" : 0,
            "done" : False
        },
        "2" : {
            "start" : 0,
            "end" : 0,
            "done" : False
        }
    }
    # Dictionary to check whether mouselab task has been completed
    mdp_timestamps = {
        "start" : 0,
        "end" : 0,
        "done": False
    }
    
    # Use state machine to parse participant trial data, update state 
    #  based on trials seen
    current_state = "stroop_1"
    stroop1Completed = 0
    stroop1Correct = 0
    stroop2Completed = 0
    stroop2Correct = 0
    last_trial = participant['data'][0]
    feedback = ""
    comments = ""
    for idx, trial in enumerate(participant['data']):
        trial_type = trial['trialdata']['trial_type']
        try:
            trial_id = str(trial['trialdata']['trial_id'])
        except KeyError:
            trial_id = ""
        
        # Start 1st stroop task
        if trial_id == "stroop_1_ready_1" and not stroop_timestamps["1"]["done"]:
            stroop_timestamps["1"]["start"] = trial['dateTime']
        
        # Start 2nd stroop task
        if trial_id == "stroop_2_ready_1" and not stroop_timestamps["2"]["done"]:
            stroop_timestamps["2"]["start"] = trial['dateTime']
        
        # Finish 1st stroop task
        if trial_id == "finish_distractor_1" and not stroop_timestamps["1"]["done"]:
            stroop_timestamps["1"]["end"] = trial['dateTime']
            stroop_timestamps["1"]["done"] = True
        
        # Finish 2nd stroop task
        if trial_id == "finish_distractor_2" and not stroop_timestamps["2"]["done"]:
            stroop_timestamps["2"]["end"] = trial['dateTime']
            stroop_timestamps["2"]["done"] = True
        
        # Start mouselab trials
        if trial_id == "mouselab_instructions_1" and not mdp_timestamps["done"]:
            mdp_timestamps["done"] = True
            mdp_timestamps["start"] = trial['dateTime']
        
        # Finish mouselab trials
        if trial_id.startswith("final_quiz"):
            mdp_timestamps["end"] = trial['dateTime']
            
        # Stroop trial encountered, count whether it belongs to the first set or second set
        if "congruent" in trial_id or "incongruent" in trial_id or "unrelated" in trial_id:
            if not stroop_timestamps["1"]["done"]:
                stroop1Completed += 1
                
                if trial['trialdata']["response"].lower() == trial['trialdata']["correct_response"].lower():
                    stroop1Correct += 1
                
            else:
                stroop2Completed += 1
                if trial['trialdata']["response"].lower() == trial['trialdata']["correct_response"].lower():
                    stroop2Correct += 1
        
        # Mouselab trial encountered
        if trial_type == 'mouselab-mdp':
            if trial_id.startswith('practice'):
                numPracTrials += 1
            else:
                trialLengths.append(trial['trialdata']['trialTime'])
                
                trialScores.append(trial['trialdata']['score'])
                rewardsWithheld.append(trial['trialdata']['withholdReward'])
                
                # Getting expected scores
                g_truth = [0.0] + trial['trialdata']['stateRewards'][1:]
                mEnv = MouselabEnv.new_symmetric_registered('high_increasing', ground_truth=g_truth)
                clicks = trial['trialdata']['queries']['click']['state']['target']
                for click in clicks:
                    mEnv.step(int(click))
                
                
                # Saving the number of clicks and the average depth of all clicks
                numClicks.append(len(clicks))
                
                # Level one nodes are 1,5,9
                clicksL1 = len([c for c in clicks if int(c) in [1,5,9]])
                clicksL2 = len([c for c in clicks if int(c) in [2,6,10]])
                clicksL3 = len([c for c in clicks if int(c) in [3,4,7,8,11,12]])
                try:
                    avgLevel = (clicksL1 + 2*clicksL2 + 3*clicksL3)/len(clicks)
                except:
                    avgLevel = 0
                    
                avgClickLevel.append(avgLevel)
                
                # plan quality is expected score of the trial minus costs
                planQuality = mEnv._term_reward() + trial['trialdata']['costs']/(1.0 if participant['condition'] == 0 else scarcity_level)
                expectedScores.append(planQuality)
                
        # Save information about end questionnaire
        if trial_type == 'survey-html-form':
            if 'effort' in trial['trialdata']['response']:
                effort = trial['trialdata']['response']['effort']
                demographicsAvailable = True
        
        # Participant failed quiz
        if trial_id.startswith("finish_fail"):
            demographicsAvailable = True
        
        # Save information about participant feedback/comments
        if trial_type == 'survey-text':
            feedback = trial['trialdata']['response']['Q0']
            comments = trial['trialdata']['response']['Q2']
        last_trial = trial

    # Check whether participant dropped out by seeing whether any of the phases doesn't have an end timestamp
    dropoutPoint = None
    for obj, type_ in zip([mdp_timestamps, stroop_timestamps["1"], stroop_timestamps["2"]], ["mdp", "s1", "s2"]):
        if obj["start"] > 0 and obj["end"] == 0:
            dropoutPoint = type_
            obj["end"] = lastTrialStamp
            
    # Save participant statistics in dictionary   
    part_stats_dict = {
        "workerId": participant['workerId'],
        "Beginhit": beginhit,
        "Endhit": endhit,
        "Beginexp": beginexp,
        "psiturkStatus" : participant["psiturkStatus"],
        "browser" : participant["browser"],
        "platform" : participant["platform"],
        "language" : participant["language"],
        "hitLength": hitLength,
        "expLength": expLength,
        "totalLengthSum": (lastTrialStamp - firstTrialStamp) / 1000,
        "numQuizAttempts": numPracTrials / 2,
        "trialLengths": trialLengths,
        "trialScores" : trialScores,
        "numTrialsCompleted": len(trialLengths),
        "rewardsWithheld": rewardsWithheld,
        "effort": effort,
        "condition": participant['condition'],
        "stroop1Length" : (stroop_timestamps["1"]["end"] - stroop_timestamps["1"]["start"]) / 1000,
        "stroop2Length" : (stroop_timestamps["2"]["end"] - stroop_timestamps["2"]["start"]) / 1000,
        "mouselabLength" : (mdp_timestamps["end"] - mdp_timestamps["start"]) / 1000,
        "stroop1Completed" : stroop1Completed,
        "stroop2Completed" : stroop2Completed,
        "stroop1Correct" : stroop1Correct,
        "stroop2Correct" : stroop2Correct,
        "dropoutPoint" : dropoutPoint,
        "feedback": feedback,
        "comments": comments,
        "expectedScores": expectedScores,
        "numClicks": numClicks,
        "avgClickLevel" : avgClickLevel,
        "demographicsAvailable": demographicsAvailable
    }
    
    # Get final score from data file, or if not present, sum up the scores of all the rewarded trials
    if 'questiondata' in participant and 'final_score' in participant['questiondata']:
        part_stats_dict['finalScore'] = participant['questiondata']['final_score']
    else:
        part_stats_dict['finalScore'] = sum([score for (score, withheld) in zip(trialScores, rewardsWithheld) if not withheld])
    
    part_stats_dict["expLengthSum"] = part_stats_dict["stroop1Length"] + part_stats_dict["mouselabLength"] + part_stats_dict["stroop2Length"]
    part_stats_dict["expLengthDiff"] = part_stats_dict["expLengthSum"] - part_stats_dict["expLength"]
    if len(trialLengths) > 0:
        part_stats_dict["averageTrialLength"] = sum(trialLengths) / len(trialLengths)
        part_stats_dict["propWithheld"] = sum(rewardsWithheld) / len(rewardsWithheld)
        
    part_stats_dicts.append(part_stats_dict)
    

In [ ]:
# Compiling all data into trial data

all_trial_data = {
    "trialScores": [],
    "expectedScores": [],
    "numClicks" : [],
    "avgClickLevel" : [],
    "trialNumbers": [],
    "scarce": [],
    "numRewardedTrials": [],
    "numUnrewardedTrials": [],
    "workerId": [],
    "rewardsWithheld" : [],
}

count = 0;

for part in part_stats_dicts:
    if len(part['trialScores']) not in [30, 120]:
        print(part['condition'], part['workerId'], "MDP Data Incomplete")
        continue
        
    # Include in analysis only those participants whose data is complete
    if not part['demographicsAvailable']:
        print(part['condition'], part['workerId'], "Demographics missing")
        continue
    count += 1
    all_trial_data['trialScores'] += part['trialScores']
    all_trial_data['rewardsWithheld'] += part['rewardsWithheld']
    all_trial_data['workerId'] += [part['workerId']] * len(part['trialScores'])
    all_trial_data['expectedScores'] += part['expectedScores']
    all_trial_data['numClicks'] += part['numClicks']
    all_trial_data['avgClickLevel'] += part['avgClickLevel']
    all_trial_data['trialNumbers'] += list(range(1,len(part['trialScores'])+1))
    all_trial_data['scarce'] += [int(len(part['trialScores']) == 120)] * len(part['trialScores'])
    all_trial_data['numRewardedTrials'] += [i - sum(part['rewardsWithheld'][0:i]) for i in range(len(part['trialScores']))]
    all_trial_data['numUnrewardedTrials'] += [sum(part['rewardsWithheld'][0:i]) for i in range(len(part['trialScores']))]

all_trials_df = pd.DataFrame(all_trial_data)


In [ ]:
all_trials_df

In [ ]:
# Data to use for the remainder of the analysis

model_save_folder = "../results/model_sim_data/"
control_model = "2.0.3.0.0"
scarce_model = "2.0.3.0.0"
num_simulations = 1
model_file_name = f"c{control_model}_s{scarce_model}_{num_simulations}.csv"

# Set this to true if the analysis is to be performed on data simulated by the model 
# (stored in the folder results/model_sim_data)
use_model_data = False

if use_model_data:
    filtered_data = pd.read_csv(model_save_folder + model_file_name)
else:
    filtered_data = all_trials_df

In [ ]:
filtered_data

# Visualizing Data

## Expected Scores

In [ ]:
# Plotting mean expected score vs. trial number for each condition

data_to_copy = filtered_data.copy() # all_trials_df, outliers_excluded, or outliers_excluded_part

num_scarce = len(data_to_copy.loc[data_to_copy.scarce == 1].groupby('workerId'))
num_control = len(data_to_copy.loc[data_to_copy.scarce == 0].groupby('workerId'))

# Taking the mean expected score over all trials 
scarce_data_all = np.array(data_to_copy.loc[data_to_copy.scarce == 1].groupby('trialNumbers').mean()["expectedScores"])
scarce_data_all_se = np.array(data_to_copy.loc[data_to_copy.scarce == 1].groupby('trialNumbers').std()["expectedScores"]) / np.sqrt(num_scarce)
control_data = np.array(data_to_copy.loc[data_to_copy.scarce == 0].groupby('trialNumbers').mean()["expectedScores"])
control_data_se = np.array(data_to_copy.loc[data_to_copy.scarce == 0].groupby('trialNumbers').std()["expectedScores"]) / np.sqrt(num_scarce)
scarce_data_first_30 = scarce_data_all[0:30]
scarce_data_first_30_se = scarce_data_all_se[0:30]

# Taking only the rewarded trials in the scarce condition
rewarded_trials_df = pd.DataFrame(columns=data_to_copy.columns)
for worker in np.unique(data_to_copy.workerId):
    workerRows = data_to_copy.loc[(data_to_copy.scarce == 1) & (data_to_copy.workerId == worker)].reset_index(drop=True)
    rewarded = workerRows.loc[workerRows.rewardsWithheld == False]
    rewarded['trialNumbers'] = list(range(1,len(rewarded)+1))
    rewarded_trials_df = pd.concat([rewarded_trials_df,rewarded], ignore_index=True)

rewarded_trials_df['trialNumbers'] = rewarded_trials_df['trialNumbers'].astype("int64")
rewarded_trials_df['scarce'] = rewarded_trials_df['scarce'].astype("int64")
rewarded_trials_df['numRewardedTrials'] = rewarded_trials_df['numRewardedTrials'].astype("int64")
rewarded_trials_df['numUnrewardedTrials'] = rewarded_trials_df['numUnrewardedTrials'].astype("int64")
rewarded_trials_df['rewardsWithheld'] = rewarded_trials_df['rewardsWithheld'].astype("boolean")
scarce_data_rewarded = np.array(rewarded_trials_df.groupby('trialNumbers').mean()["expectedScores"])
scarce_data_rewarded_sd = np.array(rewarded_trials_df.groupby('trialNumbers').std()["expectedScores"])

plt.figure()
#plt.title("Learning Curves over All Trials")
plt.plot(list(range(len(scarce_data_all))), scarce_data_all, label="Scarcity", color='darkturquoise')
plt.fill_between(list(range(len(scarce_data_all))), scarce_data_all + 1.96*scarce_data_all_se, scarce_data_all - 1.96*scarce_data_all_se, alpha=0.15, color='darkturquoise')
plt.plot(list(range(len(control_data))), control_data, label="Control", color='firebrick')
plt.fill_between(list(range(len(control_data))), control_data + 1.96*control_data_se, control_data - 1.96*control_data_se, alpha=0.15, color='firebrick')
plt.axvline(13, linestyle='--', color='darkturquoise')
plt.axvline(9, linestyle='--', color='firebrick')
#plt.axhline(39.97, color='k', label='Optimal', linestyle="--")
plt.legend(loc="lower right")
#plt.ylim([10,60])
plt.xlim([0, 130])
plt.xlabel("Trial Number")
plt.ylabel("Average expected trial score")
plt.show()

plt.figure()
plt.title("Learning Curves over Rewarded Trials")
plt.plot(list(range(len(scarce_data_rewarded))), scarce_data_rewarded, label="Scarce")
plt.plot(list(range(len(control_data))), control_data, label="Control")
plt.legend()
plt.ylim([10,60])
plt.xlim([0, 40])
plt.xlabel("Trial Number")
plt.ylabel("Average expected score")
plt.show()

plt.figure()
plt.title("Learning Curves over First 30 trials")
plt.plot(list(range(len(scarce_data_first_30))), scarce_data_first_30, label="Scarce")
plt.plot(list(range(len(control_data))), control_data, label="Control")
plt.legend()
plt.ylim([10,60])
plt.xlim([0, 40])
plt.xlabel("Trial Number")
plt.ylabel("Average expected score")
plt.show()



## Clicks

In [ ]:
# Plotting mean expected score vs. trial number for each condition

data_to_copy = filtered_data.copy() # all_trials_df, outliers_excluded, or outliers_excluded_part

# Taking the mean expected score over all trials 
scarce_level_all = np.array(data_to_copy.loc[data_to_copy.scarce == 1].groupby('trialNumbers').mean()["avgClickLevel"])
scarce_level_all_sd = np.array(data_to_copy.loc[data_to_copy.scarce == 1].groupby('trialNumbers').std()["avgClickLevel"])
control_level_all = np.array(data_to_copy.loc[data_to_copy.scarce == 0].groupby('trialNumbers').mean()["avgClickLevel"])
control_level_sd = np.array(data_to_copy.loc[data_to_copy.scarce == 0].groupby('trialNumbers').std()["avgClickLevel"])

# Taking the mean expected score over all trials 
scarce_num_all = np.array(data_to_copy.loc[data_to_copy.scarce == 1].groupby('trialNumbers').mean()["numClicks"])
scarce_num_all_sd = np.array(data_to_copy.loc[data_to_copy.scarce == 1].groupby('trialNumbers').std()["numClicks"])
control_num_all = np.array(data_to_copy.loc[data_to_copy.scarce == 0].groupby('trialNumbers').mean()["numClicks"])
control_num_sd = np.array(data_to_copy.loc[data_to_copy.scarce == 0].groupby('trialNumbers').std()["numClicks"])

plt.figure()
plt.title("Learning Curves of Number of Clicks")
plt.plot(list(range(len(scarce_num_all))), scarce_num_all, label="Scarce")
plt.fill_between(list(range(len(scarce_num_all))), scarce_num_all + scarce_num_all_sd, scarce_num_all - scarce_num_all_sd, alpha=0.1)
plt.plot(list(range(len(control_num_all))), control_num_all, label="Control")
plt.fill_between(list(range(len(control_num_all))), control_num_all + control_num_sd, control_num_all - control_num_sd, alpha=0.1)
plt.axhline(3.56, color='k', label='Optimal', linestyle="--")
plt.legend()
plt.xlim([0, 130])
plt.xlabel("Trial Number")
plt.ylabel("Average number of clicks")
plt.show()

plt.figure()
plt.title("Learning Curves of Average Click Level")
plt.plot(list(range(len(scarce_level_all))), scarce_level_all, label="Scarce")
plt.fill_between(list(range(len(scarce_level_all))), scarce_level_all + scarce_level_all_sd, scarce_level_all - scarce_level_all_sd, alpha=0.1)
plt.plot(list(range(len(control_level_all))), control_level_all, label="Control")
plt.fill_between(list(range(len(control_level_all))), control_level_all + control_level_sd, control_level_all - control_level_sd, alpha=0.1)
plt.axhline(2.968, color='k', label='Optimal', linestyle="--")
plt.legend()
plt.xlim([0, 130])
plt.xlabel("Trial Number")
plt.ylabel("Average click level")
plt.show()




## Strategy Scores

In [ ]:
# Plotting mean expected score vs. trial number for each condition

data_to_copy = filtered_data.copy() # all_trials_df, outliers_excluded, or outliers_excluded_part

scarce_trial_group = data_to_copy.loc[data_to_copy.scarce == 1].groupby('trialNumbers')
control_trial_group = data_to_copy.loc[data_to_copy.scarce == 0].groupby('trialNumbers')

# Taking the mean expected score over all trials 
scarce_data_all = np.array(scarce_trial_group.mean()["strategyScores"])
scarce_data_all_sd = np.array(scarce_trial_group.std()["strategyScores"])
control_data = np.array(control_trial_group.mean()["strategyScores"])
control_data_sd = np.array(control_trial_group.std()["strategyScores"])

# Getting the proportions of clusters
scarce_data_adaptive = np.array(scarce_trial_group["cluster"].apply(lambda x: (x==0).sum()) / scarce_trial_group["cluster"].count())
control_data_adaptive = np.array(control_trial_group["cluster"].apply(lambda x: (x==0).sum()) / control_trial_group["cluster"].count())
scarce_data_modadaptive = np.array(scarce_trial_group["cluster"].apply(lambda x: (x==1).sum()) / scarce_trial_group["cluster"].count())
control_data_modadaptive = np.array(control_trial_group["cluster"].apply(lambda x: (x==1).sum()) / control_trial_group["cluster"].count())
scarce_data_maladaptive = np.array(scarce_trial_group["cluster"].apply(lambda x: (x==2).sum()) / scarce_trial_group["cluster"].count())
control_data_maladaptive = np.array(control_trial_group["cluster"].apply(lambda x: (x==2).sum()) / control_trial_group["cluster"].count())
#control_data = np.array(data_to_copy.loc[data_to_copy.scarce == 0].groupby('trialNumbers').mean()["strategyScores"])
#control_data_sd = np.array(data_to_copy.loc[data_to_copy.scarce == 0].groupby('trialNumbers').std()["strategyScores"])

# Taking only the rewarded trials in the scarce condition
rewarded_trials_df = pd.DataFrame(columns=data_to_copy.columns)
for worker in np.unique(data_to_copy.workerId):
    workerRows = data_to_copy.loc[(data_to_copy.scarce == 1) & (data_to_copy.workerId == worker)].reset_index(drop=True)
    rewarded = workerRows.loc[workerRows.rewardsWithheld == False]
    rewarded['trialNumbers'] = list(range(1,len(rewarded)+1))
    rewarded_trials_df = pd.concat([rewarded_trials_df,rewarded], ignore_index=True)

rewarded_trials_df['trialNumbers'] = rewarded_trials_df['trialNumbers'].astype("int64")
rewarded_trials_df['scarce'] = rewarded_trials_df['scarce'].astype("int64")
rewarded_trials_df['numRewardedTrials'] = rewarded_trials_df['numRewardedTrials'].astype("int64")
rewarded_trials_df['numUnrewardedTrials'] = rewarded_trials_df['numUnrewardedTrials'].astype("int64")
rewarded_trials_df['rewardsWithheld'] = rewarded_trials_df['rewardsWithheld'].astype("boolean")
scarce_data_rewarded = np.array(rewarded_trials_df.groupby('trialNumbers').mean()["strategyScores"])
scarce_data_rewarded_sd = np.array(rewarded_trials_df.groupby('trialNumbers').std()["strategyScores"])

plt.figure()
plt.title("Strategy score Learning Curves over All Trials")
plt.plot(list(range(len(scarce_data_all))), scarce_data_all, label="Scarce")
plt.fill_between(list(range(len(scarce_data_all))), scarce_data_all + scarce_data_all_sd, scarce_data_all - scarce_data_all_sd, alpha=0.1)
plt.plot(list(range(len(control_data))), control_data, label="Control")
plt.fill_between(list(range(len(control_data))), control_data + control_data_sd, control_data - control_data_sd, alpha=0.1)
plt.axhline(np.max(np.array(strategy_scores_control_list)[:,1], axis=0), color='k', label='Optimal', linestyle="--")
plt.legend()
plt.xlim([0, 130])
plt.xlabel("Trial Number")
plt.ylabel("Average strategy score")
plt.show()

plt.figure()
plt.title("Learning Curves over Rewarded Trials")
plt.plot(list(range(len(scarce_data_rewarded))), scarce_data_rewarded, label="Scarce")
plt.plot(list(range(len(control_data))), control_data, label="Control")
plt.legend()
plt.xlim([0, 40])
plt.xlabel("Trial Number")
plt.ylabel("Average strategy score")
plt.show()

plt.figure()
plt.title("Strategy Type Change")
alpha = 0.7
plt.plot(list(range(len(scarce_data_all))), scarce_data_adaptive, label="Scarce Adaptive", color='b',alpha=alpha)
plt.plot(list(range(len(control_data))), control_data_adaptive, label="Control Adaptive",color='orange',alpha=alpha)
plt.plot(list(range(len(scarce_data_all))), scarce_data_modadaptive, label="Scarce Mod-Adaptive", color='b',alpha=alpha, linestyle="--")
plt.plot(list(range(len(control_data))), control_data_modadaptive, label="Control Mod-Adaptive",color='orange',alpha=alpha, linestyle="--")
plt.plot(list(range(len(scarce_data_all))), scarce_data_maladaptive, label="Scarce Maldaptive", color='b',alpha=alpha, linestyle=":")
plt.plot(list(range(len(control_data))), control_data_maladaptive, label="Control Maldaptive",color='orange',alpha=alpha, linestyle=":")
plt.legend()
plt.ylim([0,1])
plt.xlim([0, 130])
plt.xlabel("Trial Number")
plt.ylabel("Proportion of Trials")
plt.show()


# Comparison of Fitted Models

Comparing the results of model fitting to participant data

Before running these scripts, run the command to start the local mongo db server. For this, mongodb has to be installed locally (https://www.prisma.io/dataguide/mongodb/setting-up-a-local-mongodb-database#setting-up-mongodb-on-macos) and the anonymized data has to be downloaded and stored in the folder "/usr/local/var/mongodb".

 mongod --dbpath /usr/local/var/mongodb --logpath /usr/local/var/log/mongodb/mongo.log --fork 

The local mongodb database must have a single document for each fit of the model.

This is achieved by running the scripts jsonify_model_data.py and upload_json_to_mongo.py of repository mcl_toolbox on the fitted model .pkl files.

In [ ]:
from pymongo import MongoClient

In [ ]:
# Connect to Mongo database with model information

MONGO_URI_LOCAL="localhost:27017"

URI = MONGO_URI_LOCAL
client = MongoClient(URI)
db = client["data"]
collection = db["participants"]

In [ ]:
def get_num_params(model_doc):
    best_params = json.loads(model_doc["best_params"])
    model_num = model_doc["model"]
    num_priors = len([param for param in best_params.keys() if "prior" in param])
    num_params = len(best_params)
    if "pr_weight" in best_params.keys() and model_num[0] != "3":
        num_params -= 1
    return num_params


Features of the models:

Pseudo-rewards (feature value 1 in paper):
* p - pseudo-rewards are not used (0)
* P - pseudo-rewards are used (1) 

Fixing of initial strategy weights based on participant's inferred strategy on first trial (feature value 2 in paper)
* f - model is not fit with initial weights (0)
* F - model is fit with initial weights (1)

Whether the model construes no feedback as feedback of 0 (feature value 3 in paper)
* u - do not construe absence of feedback as feedback of 0
* U - construe absence of feedback as feedback of 0

Whether the learning rule was applied after each meta-action, or only once at the end (feature value 4 in paper)
* s - applied for each meta-action
* S - applied once at the end with the cumulative cost of all meta-actions

Ignore the object-level reward 
* i - don't ignore the object-level reward
* I - ignore the object-level reward (models of type Prioritize Self in paper)

Which signal to use as feedback
* O - environment reward (models of type Prioritize Env in paper)
* E - self-learning signal (models of type Prioritize Self in paper)
* C - weighted combination of environment reward and self-learning signal (models of type Combination in paper)


Model numbers, which consist of 4-5 digits, are translated into the combinations of features as below

In [ ]:
def update_model_name(name):
    pr = ['L','p','P', 'M'] # LVOC, Reinforce no PR, REINFORCE yes PR, REINFORCE with learn from path
    fit = ['f', 'F'] # don't fix init weights, fix init weights
    action = ['s', 'S'] # don't subtract costs from reward, subtract costs from reward
    
    model_name_map = {
        "1.1": "uiOs",
        "1.2": "UiOs",
        "2.1": "uiOS",
        "3.0": "uIE",
        "3.1": "UiC",
        "3.2": "uiC",
        "3.3": "uiE"
    }
    
    name_split = name.split(".")
    action_id = None
    if len(name_split) == 5:
        # Last digit in model name (when 5 digits) indicates whether feature value 4 is activated or not
        action_idx = name_split.pop()
        action_id = action[int(action_idx)]
        
    # First digit in model name is the model index (see next cell for model indices)
    # This determines whether feature value 1 is activated or not
    model_idx = name_split.pop(0)
    model = pr[(int(model_idx)-1)]
    
    # Second digit in model name is whether feature value 2 is activated or not
    fit_idx = name_split.pop(0)
    fit_id = fit[int(fit_idx)]
    
    # Remaining features of the model based on the remaining two digits in the model name are translated
    # using the "model name map" above
    remaining_model = model_name_map[".".join(name_split)]
    new_model = fit_id + remaining_model + (action_id if action_id else "") + model
    shortened = ''.join([c for c in new_model if c.isupper()])
    return new_model, shortened
        

In [ ]:
num_evals = 6000

model_idxs = [
    "2",
    "3"
] # 6719, 6687
# Models that don't use pseudo rewards and use pseudo rewards, respectively

fit_wts = [
    "0",
    "1"
] # fit initial weights, no/yes

# See cell above for meanings of models
models = [
    "1.1",
    "1.2",
    "2.1",
    "3.0.0",
    #"3.0.1",
    "3.1.0",
    #"3.1.1", 
    "3.2.0",
    #"3.2.1",
    "3.3.0",
    #"3.3.1"
]

all_models = []
for model_idx in model_idxs:
    for fit_wt in fit_wts:
        for model in models:
            model_identifier = model_idx + "." + fit_wt + "." + model
            all_models.append(model_identifier)

conditions = [
    "control", 
    "scarce"
]
condition_trial_nums = {
    "scarce" : 120,
    "control" : 30
}

condition_data_sets = {
    "control": all_trials_df.loc[(all_trials_df.scarce == 0)],
    "scarce": all_trials_df.loc[(all_trials_df.scarce == 1)]
}

In [ ]:
scarce_participants = list(all_trials_df.loc[all_trials_df.scarce == 1].groupby('workerId').mean().index)
control_participants = list(all_trials_df.loc[all_trials_df.scarce == 0].groupby('workerId').mean().index)


In [ ]:
updated_model_name_map = {}
for model in all_models:
    new_model, shortened = update_model_name(model)
    print(model, new_model)
    # Uncomment following line to print model features for MATLAB model comparison script
    print(' '.join(["0" if c.islower() or c == "O" else ("2" if c == "C" else "1") for c in new_model]), ";")
    updated_model_name_map[model] = shortened
    updated_model_name_map[shortened] = model

In [ ]:
# Initialize variables to store data about model fits
# For each condition, for each model, for each participant
model_average_rewards = {}
participant_BIC_scores = {}
participant_BIC_metrics = {}
total_BIC_metrics = {}
condition_BIC_metrics = {}
participant_AIC_scores = {}
total_losses = {}
best_eval_id = {}
seen_pids = {}
for condition in conditions:
    model_average_rewards[condition] = {}
    participant_BIC_scores[condition] = {}
    #participant_BIC_metrics[condition] = {}
    participant_AIC_scores[condition] = {}
    condition_BIC_metrics[condition] = {}
    total_losses[condition] = {}
    best_eval_id[condition] = {}
    seen_pids[condition] = {}
    for model in all_models:
        """
        model_average_rewards[condition][model] = {
            "scores": np.array([0.] * condition_trial_nums[condition]),
            "expected": np.array([0.] * condition_trial_nums[condition]),
            "num_parts": 0
        }
        """
        model_average_rewards[condition][model] = {
            "scores": [],
            "expected": [],
            "num_clicks": [],
            "num_parts": 0
        }
        
        participant_BIC_scores[condition][model] = {}
        total_losses[condition][model] = {}
        best_eval_id[condition][model] = []
        #participant_BIC_metrics[condition][model] = {}
        participant_AIC_scores[condition][model] = {}
        condition_BIC_metrics[condition][model] = {}
        condition_BIC_metrics[condition][model]["total_loss"] = 0
        condition_BIC_metrics[condition][model]["total_params"] = 0
        condition_BIC_metrics[condition][model]["total_m_actions"] = 0
        condition_BIC_metrics[condition][model]["num_parts"] = 0
        if model not in total_BIC_metrics:
            total_BIC_metrics[model] = {}
            total_BIC_metrics[model]["total_loss"] = 0
            total_BIC_metrics[model]["total_params"] = 0
            total_BIC_metrics[model]["total_m_actions"] = 0
            total_BIC_metrics[model]["num_parts"] = 0
        seen_pids[condition][model] = {}


In [ ]:
# Go through each condition and document

num_simulations = 30

click_cost = {
    "scarce" : 1.0,
    "control": 1.0
}


for condition in conditions:
    
    for model in all_models:
        num_parts = 0
        num_missing_r = 0
        # Find the documents corresponding to this condition and model
        print({ "condition": condition, "model": model, "num_evals": num_evals })
        docs = collection.find({ "condition": condition, "model": model, "num_evals": num_evals })
        if model not in participant_BIC_metrics:
            participant_BIC_metrics[model] = {}
        print(condition, model, updated_model_name_map[model])#, len(list(docs)))
        for doc in list(docs):
            num_params = get_num_params(doc)
            pid = doc["pid"]
            best_eval = json.loads(doc["best_eval"])
            data_set = condition_data_sets[condition]
            part_data = data_set.loc[data_set.workerId == pid]
            num_meta_actions = part_data["numClicks"].sum() + len(part_data)
        
            if pid not in seen_pids[condition][model]:
                seen_pids[condition][model][pid] = 1
            else:
                seen_pids[condition][model][pid] += 1
            
            # Calculate AIC/BIC score
            BIC = 2 * best_eval["loss"] + num_params * np.log(num_meta_actions)
            AIC = 2 * best_eval["loss"] + num_params * 2
            
            # Get the losses for the model
            all_losses = json.loads(doc["all_losses"])
            
            # Store the AIC and BIC of each run
            
            if pid in participant_BIC_scores[condition][model]:
                participant_BIC_scores[condition][model][pid].append(BIC)
                participant_AIC_scores[condition][model][pid].append(AIC)
            else:
                participant_BIC_scores[condition][model][pid] = [BIC]
                participant_AIC_scores[condition][model][pid] = [AIC]
            
            
            total_losses[condition][model][pid] = all_losses
            best_trial = all_losses.index(best_eval["loss"])
            best_eval_id[condition][model].append(best_eval["tid"])
            
            # Store metrics about each individual model for later computation 
            participant_BIC_metrics[model][pid] = {}
            participant_BIC_metrics[model][pid]["loss"] = best_eval["loss"]
            participant_BIC_metrics[model][pid]["params"] = num_params
            participant_BIC_metrics[model][pid]["m_actions"] = num_meta_actions
            participant_BIC_metrics[model][pid]["condition"] = condition
            
            # Accumulate metrics about each individual model for later computation
            # of model BIC score over all participants for a given model
            total_BIC_metrics[model]["total_loss"] += best_eval["loss"]
            total_BIC_metrics[model]["total_params"] += num_params
            total_BIC_metrics[model]["total_m_actions"] += num_meta_actions
            total_BIC_metrics[model]["num_parts"] += 1
            
            # Accumulate metrics about each individual model for later computation
            # of model BIC score over all participants for a given condition
            condition_BIC_metrics[condition][model]["total_loss"] += best_eval["loss"]
            condition_BIC_metrics[condition][model]["total_params"] += num_params
            condition_BIC_metrics[condition][model]["total_m_actions"] += num_meta_actions
            condition_BIC_metrics[condition][model]["num_parts"] += 1
            
            num_parts += 1
            # Get the simulation results for the model
            if "r" not in doc:
                #print(pid)
                #print(doc)
                num_missing_r += 1
        
            else:
                rewards = json.loads(doc["r"])
                expected = json.loads(doc["mer"])
                clicks = json.loads(doc["num_clicks"])
                
                for sim_num in range(len(expected)):
                    for trial_num in range(len(expected[sim_num])):
                        expected[sim_num][trial_num] -= clicks[sim_num][trial_num]
                
                expected = np.array(expected)
                
                # Add the average score of the simulations to the average scores for that model
                model_average_rewards[condition][model]["scores"] += list(rewards) #.append(list(rewards))
                model_average_rewards[condition][model]["expected"] += list(expected) #.append(list(expected))
                model_average_rewards[condition][model]["num_clicks"] += list(clicks) #.append(list(clicks))
                model_average_rewards[condition][model]["num_parts"] += 1
        print(num_parts)
        print(num_missing_r)



In [ ]:
# Check if there are any models missing

participants = {
    "control": control_participants,
    "scarce": scarce_participants
}
expected_num_runs = 2
total_missing = 0
for condition in conditions:
    print(condition + "\n")
    for model in all_models:
        print("\n" + model)
        for pid in participants[condition]:
            if pid not in seen_pids[condition][model]:
                print(pid, 0)
                total_missing += 1
            elif seen_pids[condition][model][pid] != expected_num_runs:
                print(pid, seen_pids[condition][model][pid])
                total_missing += 1
        

In [ ]:
# Calculate single BIC score over all participants for a single model
model_BIC = pd.DataFrame.from_dict(total_BIC_metrics, orient='index')

# Calculate single BIC score per condition
scarce_BIC = pd.DataFrame.from_dict(condition_BIC_metrics["scarce"], orient='index')
control_BIC = pd.DataFrame.from_dict(condition_BIC_metrics["control"], orient='index')

# Create dataframe of individual participant metrics for bootstrapping
participant_BIC = pd.DataFrame.from_dict(participant_BIC_metrics)

# Create dataframes of participant BIC/AIC scores
scarce_part_BIC = pd.DataFrame.from_dict(participant_BIC_scores["scarce"])
control_part_BIC = pd.DataFrame.from_dict(participant_BIC_scores["control"])
#original_BIC = pd.DataFrame.from_dict(model_BIC_scores["original"])
scarce_part_AIC = pd.DataFrame.from_dict(participant_AIC_scores["scarce"])
control_part_AIC = pd.DataFrame.from_dict(participant_AIC_scores["control"])
#original_AIC = pd.DataFrame.from_dict(model_AIC_scores["original"])


In [ ]:
# Sample a subset from a dataframe where each cell is a list of values
import random

def sample_subset(df):
    new_df = df.copy()
    for col in df.columns:
        for part in df.index:
            vals = df[col][part]
            try:
                new_df[col][part] = np.random.choice(vals)
            except:
                new_df[col][part] = np.nan
    
    return new_df
        

In [ ]:
# Sample several bootstrapped BIC datasets

start = 1574
end = 10000

# Preparing dataframe for MATLAB script SPM_bm.m
for i in range(start,end+1):
    sample_num = i+1
    scarce_sample = sample_subset(scarce_part_BIC).dropna()
    control_sample = sample_subset(control_part_BIC).dropna()
    
    csv_file_scarce = f"../results/mcrl/bic_datasets/scarce_bic_{num_evals}_{sample_num}.csv"
    csv_file_control = f"../results/mcrl/bic_datasets/control_bic_{num_evals}_{sample_num}.csv"


    # Save to CSV
    scarce_sample.to_csv(csv_file_scarce)
    control_sample.to_csv(csv_file_control)


# Analysis of Model Comparison with several bootstrapped datasets

In order to test the robustness of the results of the model comparison, the same comparison procedure (spm_BMS and spm_compare_families) was performed on several datasets of model BIC scores. Each model had a certain set of features and was fit to a particular participant's data twice. The results of these models were used to bootstrap the several datasets, and the model comparison was performed on each of the datasets.

The following section contains code to parse and compile the results of the model comparison on each of the datasets, to see which results are robust and how confident we can be of the conclusions drawn from the model comparisons.

In [ ]:
import xml.etree.ElementTree as ET
from collections import defaultdict

In [ ]:
import warnings
warnings.filterwarnings('ignore')


In [ ]:
# https://stackoverflow.com/questions/7684333/converting-xml-to-dictionary-using-elementtree

def etree_to_dict(t):
    d = {t.tag: {} if t.attrib else None}
    children = list(t)
    if children:
        dd = defaultdict(list)
        for dc in map(etree_to_dict, children):
            for k, v in dc.items():
                dd[k].append(v)
        d = {t.tag: {k: v[0] if len(v) == 1 else v
                     for k, v in dd.items()}}
    if t.attrib:
        d[t.tag].update(('@' + k, v)
                        for k, v in t.attrib.items())
    if t.text:
        text = t.text.strip()
        if children or t.attrib:
            if text:
                d[t.tag]['#text'] = text
        else:
            d[t.tag] = text
    return d

In [ ]:
# Read files

results_folder = "../results/spm_results"
results_files = os.listdir(results_folder)

metrics = ["exp_r", "xp"]

# Return a list of pandas column names for a list of data categories
# by appending the name of the metric to the categories
def get_all_df_cols_for_names(names, init_cols=[]):
    new_list = [*init_cols]
    for name in names:
        for metric in metrics:
            new_list.append(name + " " + metric)
    return new_list


# cols = {
#     "bms": get_all_df_cols_for_names(all_models, ["condition"]),
#     "comp_1": get_all_df_cols_for_names(["No Fix", "Fix"], ["condition"]),
#     "comp_2": get_all_df_cols_for_names(["None", "Zero"]),
#     "comp_3": get_all_df_cols_for_names(["Use OL", "Ignore OL"], ["condition"]),
#     "comp_4": get_all_df_cols_for_names(["No PR", "PR"], ["condition"]),
#     "comp_5": get_all_df_cols_for_names(["No ES", "ES"], ["condition"]),
#     "comp_6": get_all_df_cols_for_names(["Only OL", "Only ER", "Both"], ["condition"]),
#     "comp_7": get_all_df_cols_for_names(["Only OL", "Only ER", "ER only when no OL","Some ER Always"], ["condition"]),
# }

dataframe_dict_lists = {}

control_count = 0
scarce_count = 0
max_count_per_cond = 8800

# Converting result xml files into dataframes 
# One dataframe for each analysis - 1 individual model analysis and 7 family comparisons
for file in results_files:
    full_file_path = os.path.join(results_folder, file)
    condition = 1 if "scarce" in file else 0
    try:
        etree = ET.parse(full_file_path)
    except:
        print(file)
        continue
        
    if max(condition * scarce_count, (1-condition) * control_count) >= max_count_per_cond:
        continue
    #print(ET.dump(etree.getroot()))
    
    control_count += 1-condition
    scarce_count += condition
    xml_dict = etree_to_dict(etree.getroot())['struct']
    #print(xml_dict)
    for analysis, outcome in xml_dict.items():
        if analysis not in dataframe_dict_lists:
            dataframe_dict_lists[analysis] = []
        analysis_dict = {
            "condition": condition
        }
        # Individual model comparison
        if analysis == "bms":
            for idx, model in enumerate(outcome["models"]):
                model_name = model.replace("x", "").replace("_", ".")
                relevant_cols = get_all_df_cols_for_names([model_name])
                for relevant_col in relevant_cols:
                    for metric in metrics:
                        if (" " + metric) in relevant_col:
                            analysis_dict[relevant_col] = float(outcome[metric][idx])
                
        # Family comparisons
        elif analysis.startswith("comp"):
            for idx, family_name in enumerate(outcome["names"]):
                relevant_cols = get_all_df_cols_for_names([family_name])
               
                for relevant_col in relevant_cols:
                    for metric in metrics:
                        if (" " + metric) in relevant_col:
                            #print(metric, relevant_col, outcome[metric][idx])
                            analysis_dict[relevant_col] = float(outcome[metric][idx])
                
        dataframe_dict_lists[analysis].append(analysis_dict)
    

# Create one dataframe for each analysis
# bms - individual model analysis
# comp1 - family comparison of fixing of strategy weights (feature value 2 = 1 vs. 0)
# comp2 - family comparison of feature value 3 = 1 vs.0 
# comp3 - family comparison of models that use environment reward and models that dont
# comp4 - family comparison of models that use pseudo rewards vs. those that don't (fv 1 = 1 vs. 0)
# comp5 - family comparison of models that use self-generated reward vs those that don't
# comp6 - family comparison of models that exclusively use one signal (environment or self-generated rewards) or both
# comp7 - family  comparison of how models use learning signals - only env reward, only self-generated, self-generated when env reward is absent, or always in combination
dataframes = {key: pd.DataFrame.from_dict(value) for key, value in dataframe_dict_lists.items()}
    
    


Descriptions of family names:

* comp_1
    * Fix - Initial weight priors are fixed by participant's initial strategy
    * No Fix - Initial weight priors are standard normal distributions
* comp_2
    * None - Absent reward construed as such
    * Zero - Absent rewrad construed as reward of zero
* comp_3
    * Ignore OL - Ignore the environment reward when learning
    * Use OL - Use the environment reward to learn where present
* comp_4
    * No PR - No pseudo rewards used
    * PR - Pseudo rewards used
* comp_5
    * No ES - self-generated expected reward not used
    * ES - self-generated expected reward used
* comp_6
    * Only OL - Uses only the environment reward for learning
    * Only ER - Uses only the self-generated expected reward for learning
    * Both - Uses both of the above for learning
* comp_7
    * Only OL - Uses only the environment reward for learning
    * Only ER - Uses only the self-generated expected reward for learning
    * ER only when no OL - Uses the self-generated expected reward for learning when there is no env reward
    * Some ER always - Always uses self-generated expected reward in combination with the env reward (if present) 

In [ ]:
analysis_descriptions = {
    "bms": "Individual model analysis",
    "comp_1": "Fixing initial weights",
    "comp_2": "Absent reward = reward 0?",
    "comp_3": "Use environment reward?",
    "comp_4": "Use Pseudo Reward?",
    "comp_5": "Use Expected Reward?",
    "comp_6": "Exclusive use of signals",
    "comp_7": "How Expected Reward is used"
}

In [ ]:
# Creating and plotting 95% CIs for the outcomes of all analyses

fig, axes = plt.subplots(len(dataframes), len(conditions), figsize=(7 * len(conditions), 7 * len(dataframes)))
for an_idx, (analysis, df) in enumerate(dataframes.items()):
    control = df.loc[df.condition == 0]
    scarce = df.loc[df.condition == 1]
    
    for cond_idx, condition in enumerate(conditions):
        ax = axes[an_idx][cond_idx]
        for metric in metrics:
            relevant_columns = [column for column in df.columns if (" " + metric) in column]
            relevant_df = df.loc[df.condition == cond_idx][relevant_columns]
            mean = relevant_df.mean()
            sd = relevant_df.std()
            # confidence level
            z = 1.96
            
            num_samples = len(relevant_df)
            error_margin = z * sd / np.sqrt(num_samples)
            if analysis != "bms":# and metric == "exp_r":
                # Printing mean statistics and confidence intervals
                print(condition, analysis)
                print("\t",list(relevant_df.columns))
                print("M\t",np.array(mean))
                print("LB\t",np.array(mean) - np.array(error_margin))
                print("UB\t", np.array(mean) + np.array(error_margin))
                print("E\t", np.array(error_margin))
                print("\n")
            
            # Printing aggregate statistics of proportions of participants who use
            # expected reward on trials with feedback vs. those without
            # with 95% CI
            if analysis == "comp_7" and metric == "exp_r":
                sum1_columns = ["Only ER exp_r", "ER only when no OL exp_r", "Some ER Always exp_r"]
                sums1 = np.array(relevant_df[sum1_columns].fillna(0)).sum(axis=1)
                sum1_mean = sums1.mean()
                sum1_sd = sums1.std()
                sum1_err = z* sum1_sd / np.sqrt(num_samples)
                sum2_columns = ["Only ER exp_r", "Some ER Always exp_r"]
                sums2 = np.array(relevant_df[sum2_columns].fillna(0)).sum(axis=1)
                sum2_mean = sums2.mean()
                sum2_sd = sums2.std()
                sum2_err = z* sum2_sd / np.sqrt(num_samples)
                print(condition, "Proportion of participants using ER when no feedback")
                print("M\t", sum1_mean)
                print("LB\t", sum1_mean-sum1_err)
                print("UB\t", sum1_mean+sum1_err)
                print("E\t", sum1_err)
                print(condition, "Proportion of participants using ER even with feedback")
                print("M\t", sum2_mean)
                print("LB\t", sum2_mean-sum2_err)
                print("UB\t", sum2_mean+sum2_err)
                print("E\t", sum2_err)
                
            #print(mean)
            axis_labels = [" ".join(col.split(" ")[:-1]) for col in relevant_columns]
            ax.scatter(axis_labels,mean, label=metric)
            ax.errorbar(axis_labels,mean, yerr=error_margin,fmt='o',capsize=4)
            if analysis == "bms":
                ax.set_xticklabels(axis_labels,rotation=90,fontsize=9)
            else:
                ax.set_xticklabels(axis_labels,rotation=45,fontsize=14)
        ax.legend()
        ax.set_title(f"{analysis_descriptions[analysis]}, {condition}, {len(relevant_df)}", fontsize=14)
fig.subplots_adjust(hspace=0.3)
plt.show()

In [ ]:
# Checking qualitative results - how frequently is a family the winning family?
# A family is the winning family if the exceedance probability is above 75%

xp_threshold = 0.75

for analysis, df in dataframes.items():
    if analysis == "bms": 
        continue
    for idx, condition in enumerate(["control", "scarce"]):
        relevant_df = df.loc[df.condition == idx].copy()
        xp_cols = [col for col in relevant_df.columns if (" xp" in col)]
        relevant_df = relevant_df[xp_cols]
        relevant_df = relevant_df.where(relevant_df > xp_threshold, 0)
        relevant_df = relevant_df.where(relevant_df == 0, 1)
        print("\n",analysis, condition, len(relevant_df))
        print([" ".join(col.split(" ")[:-1]) for col in relevant_df.columns])
        print(relevant_df.sum().values)
        


In [ ]:
# Prettier graph of results of last comparison for the paper

df = dataframes["comp_7"]
control_df = df.loc[df.condition == 0]
scarce_df = df.loc[df.condition == 1]

relevant_columns = [col for col in scarce_df.columns if (" exp_r" in col)]

meanc = control_df[relevant_columns].mean()
means = scarce_df[relevant_columns].mean()

sdc = control_df[relevant_columns].std()
sds = scarce_df[relevant_columns].std()

# confidence level
z = 1.96

num_samples = len(control_df)
errc = z * sdc / np.sqrt(num_samples)
errs = z * sds / np.sqrt(num_samples)

relevant_columns = [col for col in scarce_df.columns if (" exp_r" in col)]

plt.figure(figsize=(3,3))
axis_labels = ["V", "PS", "PE", "C"]
plt.scatter(axis_labels,meanc, label="Control",color='firebrick',marker="^", s=40)
plt.scatter(axis_labels,means, label="Scarcity",color='darkturquoise', s=40)
#plt.errorbar(axis_labels,meanc, yerr=errc,capsize=4,fmt='o', color='firebrick')
#plt.errorbar(axis_labels,means, yerr=errs,capsize=4,fmt='o', color='darkturquoise')
plt.ylabel('Prop. of Ps Explained')
plt.xlabel('Model Type')
plt.ylim([0,1])

plt.legend()
plt.show()


# 1.10 - Bar Chart of Model BMS Comparisons

In [ ]:
# Reading the BMS results

cbms_file = "../results/mcrl/cBMS_results.csv"
sbms_file = "../results/mcrl/sBMS_results.csv"

cbms_rows = []
sbms_rows = []

with open(cbms_file, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        cbms_rows.append(row)
        
with open(sbms_file, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        sbms_rows.append(row)
        
cbms_rows = [row[1:] for row in cbms_rows[0:3]]
sbms_rows = [row[1:] for row in sbms_rows[0:3]]

cbms_rows[1] = [float(val) for val in cbms_rows[1]]
cbms_rows[2] = [float(val) for val in cbms_rows[2]]

sbms_rows[1] = [float(val) for val in sbms_rows[1]]
sbms_rows[2] = [float(val) for val in sbms_rows[2]]

In [ ]:
# Summing up proportions of both conditions

import copy, functools

summed_props = copy.deepcopy(cbms_rows)

for i in range(len(summed_props[0])):
    summed_props[1][i] += sbms_rows[1][sbms_rows[0].index(summed_props[0][i])]
    
# Sort models according to summed proportions

model_props = [(x, y) for x,y in zip(summed_props[0], summed_props[1])] 

def sort_fn(a, b):
    if b[1] > a[1]:
        return 1
    elif b[1] < a[1]:
        return -1
    else:
        return 0

sorted_models = sorted(model_props, key=functools.cmp_to_key(sort_fn))

In [ ]:
def convert_model_name(name):
    # Remove x
    name = name[1:]
    
    model_type = None
    features = [0,0,0,0]
    name_split = name.split("_")
    # PR
    model = name_split.pop(0)
    features[0] = int(model) - 2
    
    # Fix wts
    fix_wts = name_split.pop(0)
    features[1] = int(fix_wts)
    
    if len(name_split) == 3:
        # Subtract costs
        sub = name_split.pop()
        features[3] = int(sub)
        
        # Remaining model
        remaining_model = '_'.join(name_split)
        if remaining_model == "3_0":
            model_type = "PS"
            features[2] = 0
        elif remaining_model == "3_1":
            model_type = "C"
            features[2] = 1
        elif remaining_model == "3_2":
            model_type = "C"
            features[2] = 0
        elif remaining_model == "3_3":
            model_type = "PE"
            features[2] = 0
    else:
        # Remaining model
        remaining_model = '_'.join(name_split)
        if remaining_model == "1_1":
            model_type = "V"
            features[2] = 0
            features[3] = 0
        elif remaining_model == "1_2":
            model_type = "V"
            features[2] = 1
            features[3] = 0
        elif remaining_model == "2_1":
            model_type = "V"
            features[2] = 0
            features[3] = 1
    
    features = [str(f) for f in features]
    return model_type+", " + "".join(features)
        
        
    
    

In [ ]:
for model, prop in sorted_models:
    print("{}\t{}".format(model, convert_model_name(model)))

In [ ]:
# Create stacked bar plot

num_models_to_plot = 38# len(sorted_models)

control_props = [cbms_rows[1][cbms_rows[0].index(model)] for model, prop in sorted_models[0:num_models_to_plot]]
scarce_props = [sbms_rows[1][sbms_rows[0].index(model)] for model, prop in sorted_models[0:num_models_to_plot]]
labels = [convert_model_name(model) for model,prop in sorted_models[0:num_models_to_plot]]
xpos = [2.5*i for i in range(len(labels))][::-1]

print(sum(control_props) + sum(scarce_props))

fig = plt.figure(figsize=(6,6.5))
width = 2.0
xlim = 0.2
# plt.barh(labels, control_props, width, label='Control')
# plt.barh(labels, scarce_props, width, label='Scarce', left=control_props)
for i in range(len(control_props)):
    plt.barh(xpos[i], control_props[i], height=width,label='Control', color='firebrick', align='edge')
    bar = plt.barh(xpos[i], scarce_props[i], height=width,label='Scarce', left=control_props[i], color='darkturquoise', align='edge')
    bar = list(bar)[0]
    w, h = bar.get_width(), bar.get_height()
    x0, y0 = bar.xy
    right_coords = x0 + w + 0.002, y0 + h/2
    plt.text(*right_coords, labels[i], va='center', fontsize=8)#,bbox=dict(facecolor='none', edgecolor='red'))
#     bar1 = plt.barh(xpos[i], control_props[i], height=width,label='Control', color='firebrick', align='edge')
#     bar2 = plt.barh(xpos[i], scarce_props[i], height=width,label='Scarce', left=xlim-scarce_props[i], color='darkturquoise', align='edge')
#     bar1 = list(bar1)[0]
#     w, h = bar1.get_width(), bar1.get_height()
#     x0, y0 = bar1.xy
#     right_coords = x0 + w + 0.002, y0 + h/2
#     plt.text(xlim/2, y0+h/2, labels[i], ha='center', va='center', fontsize=8)#,bbox=dict(facecolor='none', edgecolor='red'))
plt.legend(['Control', 'Scarcity'], prop={'size': 12})
plt.tick_params(axis='y', which='both', left=False, right=False, labelleft=False)
plt.xticks(np.linspace(0,0.2,5))
plt.xlabel("Prop. of participants in condition best explained")
plt.ylabel("Model")
plt.xlim([0, xlim])

plt.show()

# End of Analyses Reported in the Paper

The model comparison takes place using the MATLAB script SPM_bm.m. The remainder of this notebook is irrelevant to the results reported in the paper.

In [ ]:
# Plotting the model performance against participant performance

import warnings
warnings.filterwarnings('ignore')


column = "expected" #expected, scores
df_column = "expectedScores" #trialScores, expectedScores


fig, ax = plt.subplots(nrows=len(all_models),ncols=len(conditions),figsize=(5 * len(conditions), 3 * len(all_models)),squeeze=False)
ax = ax.flatten()

show_std = False

plot_idx = 0

model_label_map = {
    
}

for model in all_models:
    updated_name = updated_model_name_map[model]
    for condition in conditions:
        model_exp_c = np.array(model_average_rewards[condition][model][column])
        avg = model_exp_c.mean(axis=0)
        std = model_exp_c.std(axis=0)
        part_data = condition_data_sets[condition]
        print(condition, len(part_data))
        c_data = np.array(part_data.loc[(part_data.workerId.isin(seen_pids[condition][model]))].groupby('trialNumbers').mean()[df_column])
        c_data_sd = np.array(part_data.loc[(part_data.workerId.isin(seen_pids[condition][model]))].groupby('trialNumbers').std()[df_column])
        try:
            ax[plot_idx].plot(range(len(avg)), avg, label="model", color='blue')
            if show_std:
                ax[plot_idx].fill_between(
                    list(range(len(avg))),
                    avg + std,
                    avg - std,
                    alpha=0.1
                )
            ax[plot_idx].plot(range(len(c_data)), c_data, label="participants", color='orange')
            if show_std:
                ax[plot_idx].fill_between(
                    list(range(len(c_data))),
                    c_data + c_data_sd,
                    c_data - c_data_sd,
                    alpha=0.1
                )
            ax[plot_idx].legend()
        except:
            pass
        ax[plot_idx].set_title(f"{updated_name} - {condition}")
    
        plot_idx += 1
    


plt.subplots_adjust(
                    wspace=0.4, 
                    hspace=0.4)
plt.show()

In [ ]:
# Create mapping between models and which family they belong

model_name_update = ["x"+"_".join(m.split(".")) for m in all_models]
model_family_map = {}

for m, um in zip(all_models, model_name_update):
    model_family_map[m] = convert_model_name(um).split(",")[0]
    
model_families = list(np.unique(list(model_family_map.values())))

In [ ]:
np.array(model_average_rewards["scarce"]["2.0.1.1"]["expected"]).shape

In [ ]:
import warnings
warnings.filterwarnings('ignore')


column = "expected" #expected, scores
df_column = "expectedScores" #trialScores, expectedScores


fig, ax = plt.subplots(nrows=1,ncols=len(conditions),figsize=(5 * len(conditions), 3),squeeze=False)
ax = ax.flatten()

show_std = False

colors = ["orange", "red", "green", "gold"]

for plot_idx, condition in enumerate(conditions):
    part_data = condition_data_sets[condition]
    N = len(part_data) / np.max(part_data.trialNumbers)
    print(condition, N)
    c_data = np.array(part_data.groupby('trialNumbers').mean()[df_column])
    c_data_se = np.array(part_data.groupby('trialNumbers').std()[df_column]) / np.sqrt(N) 
    ax[plot_idx].plot(range(len(c_data)), c_data, label="participants", color='orange')
    if show_std:
        ax[plot_idx].fill_between(
            list(range(len(c_data))),
            c_data + c_data_sd,
            c_data - c_data_sd,
            alpha=0.1
        )
    for fidx, family in enumerate(model_families):
        rel_model_list = [m for m in all_models if model_family_map[m]==family]
        family_total_rewards = None
        total_rew = 0
        for m in rel_model_list:
            print(condition, m, column)
            mrew = model_average_rewards[condition][m][column]
            if family_total_rewards is None:
                family_total_rewards = np.array(mrew)
            else:
                family_total_rewards = np.append(family_total_rewards, np.array(mrew), axis=0)
            
        avg = family_total_rewards.mean(axis=0)
        std = family_total_rewards.std(axis=0) / np.sqrt(family_total_rewards.shape[0])
        print(family, family_total_rewards.shape[0])
        
        
        

In [ ]:
# Plotting model losses

import warnings
warnings.filterwarnings('ignore')


fig, ax = plt.subplots(nrows=len(all_models),ncols=len(conditions),figsize=(5 * len(conditions), 3 * len(all_models)),squeeze=False)
ax = ax.flatten()

# Set to true to also see confidence interval on true average 
show_std = True

plot_idx = 0

model_label_map = {
    
}

for model in all_models:
    updated_name = updated_model_name_map[model]
    for condition in conditions:
        all_losses = total_losses[condition][model]
        losses_list = [value for key,value in all_losses.items() if len(value) == num_evals]
        losses = np.array(losses_list)
        try:
            avg = losses.mean(axis=0)
            std = losses.std(axis=0) / np.sqrt(losses.shape[0])
        except Exception as e:
            avg = losses.mean(axis=0)
            std = losses.std(axis=0) / np.sqrt(losses.shape[0])
        try:
            ax[plot_idx].plot(range(len(avg)), avg, label="model", color='blue')
            if show_std:
                ax[plot_idx].fill_between(
                    list(range(len(avg))),
                    avg + 1.96 * std,
                    avg - 1.96 * std,
                    alpha=0.9
                )
        except:
            pass
        ax[plot_idx].set_title(f"{updated_name} - {model} - {condition}")
    
        plot_idx += 1
    


plt.subplots_adjust(
                    wspace=0.4, 
                    hspace=0.4)
plt.show()

In [ ]:
# Observing size of individual loss improvement

import warnings
warnings.filterwarnings('ignore')


fig, ax = plt.subplots(nrows=len(all_models),ncols=len(conditions),figsize=(5 * len(conditions), 3 * len(all_models)),squeeze=False)
ax = ax.flatten()

show_std = False

plot_idx = 0

model_label_map = {
    
}

for model in all_models:
    updated_name = updated_model_name_map[model]
    for condition in conditions:
        all_losses = total_losses[condition][model]
        losses_list = [value for key,value in all_losses.items() if len(value) == num_evals]
        losses = np.array(losses_list)
        loss_improvements = []
        for part_loss in losses:
            part_imp = []
            best_loss = part_loss[0]
            for loss in part_loss[1:]:
                part_imp.append(max(0, best_loss - loss))
                if loss < best_loss:
                    best_loss = loss
            loss_improvements.append(part_imp)
        loss_improvements = np.array(loss_improvements)
        try:
            avg = loss_improvements.mean(axis=0)
            std = loss_improvements.std(axis=0) / np.sqrt(loss_improvements.shape[0])
        except:
            avg = loss_improvements.mean(axis=0)
            std = loss_improvements.std(axis=0) / np.sqrt(loss_improvements.shape[0])
        try:
            ax[plot_idx].plot(range(len(avg)), avg, label="model", color='blue')
            if show_std:
                ax[plot_idx].fill_between(
                    list(range(len(avg))),
                    avg + 1.96 * std,
                    avg - 1.96 * std,
                    alpha=0.9
                )
        except:
            pass
        ax[plot_idx].set_title(f"{updated_name} - {model} - {condition}")
    
        plot_idx += 1
    


plt.subplots_adjust(
                    wspace=0.4, 
                    hspace=0.4)
plt.show()

In [ ]:
# Observing evolution of best loss

import warnings
warnings.filterwarnings('ignore')


fig, ax = plt.subplots(nrows=len(all_models),ncols=len(conditions),figsize=(5 * len(conditions), 3 * len(all_models)),squeeze=False)
ax = ax.flatten()

show_std = True

plot_idx = 0

model_label_map = {
    
}

for model in all_models:
    updated_name = updated_model_name_map[model]
    for condition in conditions:
        all_losses = total_losses[condition][model]
        losses_list = [value for key,value in all_losses.items() if len(value) == num_evals]
        losses = np.array(losses_list)
        loss_improvements = []
        for part_loss in losses:
            part_imp = []
            best_loss = part_loss[0]
            for loss in part_loss[1:]:
                if loss < best_loss:
                    best_loss = loss
                part_imp.append(best_loss)
            loss_improvements.append(part_imp)
        loss_improvements = np.array(loss_improvements)
        try:
            avg = loss_improvements.mean(axis=0)[14500:]
            std = loss_improvements.std(axis=0)[14500:] / np.sqrt(loss_improvements.shape[0])
            range_diff = "{0:0.3f}".format(avg[0] - avg[-1])
        except:
            avg = loss_improvements.mean(axis=0)
            std = loss_improvements.std(axis=0) / np.sqrt(loss_improvements.shape[0])
            range_diff = "0"
        
        try:
            ax[plot_idx].plot(range(len(avg)), avg, label="model", color='blue')
            if show_std:
                ax[plot_idx].fill_between(
                    list(range(len(avg))),
                    avg + 1.96 * std,
                    avg - 1.96 * std,
                    alpha=0.5
                )
        except:
            pass
        ax[plot_idx].set_title(f"{updated_name} - {model} - {condition}: {range_diff}")
    
        plot_idx += 1
    


plt.subplots_adjust(
                    wspace=0.4, 
                    hspace=0.4)
plt.show()

In [ ]:

# Histogram of the trial ID where the best loss was achieved

import warnings
warnings.filterwarnings('ignore')


fig, ax = plt.subplots(nrows=len(all_models),ncols=len(conditions),figsize=(5 * len(conditions), 3 * len(all_models)),squeeze=False)
ax = ax.flatten()

show_std = False

plot_idx = 0

model_label_map = {
    
}

for model in all_models:
    updated_name = updated_model_name_map[model]
    for condition in conditions:
        best_trials = best_eval_id[condition][model]
        ax[plot_idx].hist(best_trials)
        avg_trial_num = float("{0:0.3f}".format(np.mean(best_trials)))
        ax[plot_idx].set_title(f"{updated_name} - {model} - {condition}: {avg_trial_num}")
    
        plot_idx += 1
    


plt.subplots_adjust(
                    wspace=0.4, 
                    hspace=0.4)
plt.show()

In [ ]:
#l = np.sort(losses.mean(axis=0))[::-1]
y = losses.mean(axis=0)
best_loss = y[0]
l = [best_loss]

for loss in y[1:]:
    if loss < best_loss:
        best_loss = loss
    l.append(best_loss)

plt.plot(range(len(l)), l)
plt.xlabel("Evaluation #")
plt.ylabel("Best loss")
plt.show()

In [ ]:
# Comparing BIC scores of models

metric = "mean" # average, mode

if metric == "mean":
    s_BIC = list(scarce_part_BIC.mean().sort_values().index)
    s_BIC = [updated_model_name_map[m] for m in s_BIC]
    c_BIC = list(control_part_BIC.mean().sort_values().index)
    c_BIC = [updated_model_name_map[m] for m in c_BIC]
    #o_BIC = list(original_BIC.mean().sort_values().index)
    s_AIC = list(scarce_part_AIC.mean().sort_values().index)
    s_AIC = [updated_model_name_map[m] for m in s_AIC]
    c_AIC = list(control_part_AIC.mean().sort_values().index)
    c_AIC = [updated_model_name_map[m] for m in c_AIC]
    #o_AIC = list(original_AIC.mean().sort_values().index)
    
    print("Best models by average")
    print("Scarce BIC - Best Models")
    print(s_BIC)
    print("Control BIC - Best Models")
    print(c_BIC)
#     print("Original BIC - Best Models")
#     print(o_BIC)
    print("Scarce AIC - Best Models")
    print(s_AIC)
    print("Control AIC - Best Models")
    print(c_AIC)
#     print("Original AIC - Best Models")
#     print(o_BIC)

elif metric == "mode":
    s_BIC = list(scarce_part_BIC.idxmin(axis=1).value_counts().index)
    s_BIC = [updated_model_name_map[m] for m in s_BIC]
    c_BIC = list(control_part_BIC.idxmin(axis=1).value_counts().index)
    c_BIC = [updated_model_name_map[m] for m in c_BIC]
    #o_BIC = list(original_BIC.idxmin(axis=1).value_counts().index)
    s_AIC = list(scarce_part_AIC.idxmin(axis=1).value_counts().index)
    s_AIC = [updated_model_name_map[m] for m in s_AIC]
    c_AIC = list(control_part_AIC.idxmin(axis=1).value_counts().index)
    c_AIC = [updated_model_name_map[m] for m in c_AIC]

    print("Best models by mode")
    print("Scarce BIC - Best Models")
    print(s_BIC)
    print("Control BIC - Best Models")
    print(c_BIC)
#     print("Original BIC - Best Models")
#     print(o_BIC)
    print("Scarce AIC - Best Models")
    print(s_AIC)
    print("Control AIC - Best Models")
    print(c_AIC)
    

In [ ]:
def print_model_series(series):
    models = series.index
    vals = list(series)
    for model, val in zip(models, vals):
        print("\t{0}\t{1:0.5f}\t{2}".format(updated_model_name_map[model],val,model))

In [ ]:
# Comparing single BIC score for each model

model_BIC["BIC"] = 2 * model_BIC["total_loss"] + model_BIC["total_params"] * np.log(model_BIC["total_m_actions"])
model_BIC["AIC"] = 2 * model_BIC["total_loss"] + model_BIC["total_params"] * 2

print("BIC:")
print_model_series(model_BIC["BIC"].sort_values(ascending=True)[0:15])

print("\nAIC:")
print_model_series(model_BIC["AIC"].sort_values(ascending=True)[0:15])

In [ ]:
# Comparing single BIC score for each model for each condition

scarce_BIC["BIC"] = 2 * scarce_BIC["total_loss"] + scarce_BIC["total_params"] * np.log(scarce_BIC["total_m_actions"])
scarce_BIC["AIC"] = 2 * scarce_BIC["total_loss"] + scarce_BIC["total_params"] * 2

print("Scarce BIC:")
print_model_series(scarce_BIC["BIC"].sort_values()[0:10])

print("\nScarce AIC:")
print_model_series(scarce_BIC["AIC"].sort_values()[0:10])

control_BIC["BIC"] = 2 * control_BIC["total_loss"] + control_BIC["total_params"] * np.log(control_BIC["total_m_actions"])
control_BIC["AIC"] = 2 * control_BIC["total_loss"] + control_BIC["total_params"] * 2

print("Control BIC:")
print_model_series(control_BIC["BIC"].sort_values()[0:10])

print("\nControl AIC:")
print_model_series(control_BIC["AIC"].sort_values()[0:10])

In [ ]:
# Bootstrapping the BIC/AIC confidence intervals for all participants taken together
import random

models_to_bootstrap = [
    '2.0.1.1',
    '2.0.1.2',
    '2.0.2.1',
    '2.0.3.0.0',
    '2.0.3.0.1',
    '2.0.3.1.0',
    '2.0.3.2.0',
    '2.0.3.3.0',
    '3.0.1.1',
    '3.0.1.2',
    '3.0.2.1',
    '3.0.3.0.0',
    '3.0.3.1.0',
    '3.0.3.2.0',
    '3.0.3.3.0',
]
    

num_samples = 1000

sample_size = len(participant_BIC)
bootstrapped_BICs = {}
bootstrapped_AICs = {}

bdifference_matrix = np.zeros((len(models_to_bootstrap), len(models_to_bootstrap), num_samples))
adifference_matrix = np.zeros((len(models_to_bootstrap), len(models_to_bootstrap), num_samples))

for sample_num in range(num_samples):
    if (sample_num+1) % 100 == 0: print(sample_num+1)
    sample_ids = np.random.choice(range(0,sample_size), size=(sample_size,), replace=True)
    for idx, model in enumerate(models_to_bootstrap):
        relevant_column = participant_BIC[model]
        if model not in bootstrapped_BICs:
            bootstrapped_BICs[model] = []
        if model not in bootstrapped_AICs:
            bootstrapped_AICs[model] = []


        samples = relevant_column[sample_ids]
        for sample in samples:
            # Random sample with replacement
            
            %timeit sample = relevant_column.sample().values[0]
            sample_loss += sample["loss"]
            sample_params += sample["params"]
            sample_actions += sample["m_actions"]
        sample_BIC = 2 * sample_loss + sample_params * np.log(sample_actions)
        sample_AIC = 2 * sample_loss + sample_params * 2
        bootstrapped_BICs[model].append(sample_BIC)
        bootstrapped_AICs[model].append(sample_AIC)
        
        # Subtract and add as needed to compute the difference in the scores between
        # all the models for this sample
        bdifference_matrix[idx,:,sample_num] += sample_BIC
        bdifference_matrix[:,idx,sample_num] -= sample_BIC
        adifference_matrix[idx,:,sample_num] += sample_AIC
        adifference_matrix[:,idx,sample_num] -= sample_AIC
        
    
    

In [ ]:
# Calculate 95% CIs for each model

model_BIC_CIs = {}
model_AIC_CIs = {}

for model in models_to_bootstrap:
    bmean = np.mean(bootstrapped_BICs[model])
    bstd = np.std(bootstrapped_BICs[model])
    amean = np.mean(bootstrapped_AICs[model])
    astd = np.std(bootstrapped_AICs[model])
    
    model_BIC_CIs[model] = (bmean - 2 * bstd, bmean + 2 * bstd)
    model_AIC_CIs[model] = (amean - 2 * astd, amean + 2 * astd)

In [ ]:
# Plotting histograms of BICs
models_to_plot = [
    "2.0.3.0.0",
    "2.0.2.1",
    #"2.0.3.2.0",
    #"3.1.1.1"
]

min_BIC = float("inf")
max_BIC = float("-inf")

min_AIC = float("inf")
max_AIC = float("-inf")

for model in models_to_plot:
    model_min_BIC = min(bootstrapped_BICs[model])
    model_min_AIC = min(bootstrapped_AICs[model])
    model_max_BIC = max(bootstrapped_BICs[model])
    model_max_AIC = max(bootstrapped_AICs[model])
    
    if model_min_BIC < min_BIC:
        min_BIC = model_min_BIC
    
    if model_min_AIC < min_AIC:
        min_AIC = model_min_AIC
        
    if model_max_BIC > max_BIC:
        max_BIC = model_max_BIC
        
    if model_max_AIC > max_AIC:
        max_AIC = model_max_AIC
        
BIC_bins = np.linspace(min_BIC - 1000, max_BIC + 1000, 100)
AIC_bins = np.linspace(min_AIC - 1000, max_AIC + 1000, 100)

colors = ['orange', 'blue', 'green', 'yellow']

for idx, model in enumerate(models_to_plot):
    plt.figure(1)
    plt.hist(bootstrapped_BICs[model], BIC_bins, alpha=0.5, label=model, color=colors[idx])
    plt.axvline(model_BIC_CIs[model][1], linestyle='--', color=colors[idx])
    plt.axvline(model_BIC_CIs[model][0], linestyle='-.', color=colors[idx])
    plt.figure(2)
    plt.hist(bootstrapped_AICs[model], AIC_bins, alpha=0.5, label=model, color=colors[idx])
    plt.axvline(model_AIC_CIs[model][1], linestyle='--', color=colors[idx])
    plt.axvline(model_AIC_CIs[model][0], linestyle='-.', color=colors[idx])
plt.figure(1)
plt.legend()
plt.figure(2)
plt.legend()

plt.show()

In [ ]:
# Bootstrapping the BIC/AIC confidence intervals for conditions separately
import random

models_to_bootstrap = [
    '2.0.1.1',
    '2.0.1.2',
    '2.0.2.1',
    '2.0.3.0.0',
    '2.0.3.0.1',
    '2.0.3.1.0',
    '2.0.3.2.0',
    '2.0.3.3.0',
    '3.0.1.1',
    '3.0.1.2',
    '3.0.2.1',
    '3.0.3.0.0',
    '3.0.3.1.0',
    '3.0.3.2.0',
    '3.0.3.3.0',
]
    

num_samples = 1000


bootstrapped_BICs_conds = {}
bootstrapped_AICs_conds = {}

for condition in conditions:
    condition_BICs = participant_BIC.loc[participant_BIC.index.isin(condition_data_sets[condition].workerId)]
    bootstrapped_BICs_conds[condition] = {}
    bootstrapped_AICs_conds[condition] = {}
    sample_size = len(condition_BICs)
    for model in models_to_bootstrap:
        relevant_column = condition_BICs[model]
        print(condition, model)
        bootstrapped_BICs_conds[condition][model] = []
        bootstrapped_AICs_conds[condition][model] = []

        for i in range(num_samples):
            if (i+1) % 100 == 0: print(i+1)
            sample_loss = 0
            sample_params = 0
            sample_actions = 0

            samples = relevant_column.sample(sample_size, replace=True)
            for sample in samples:
                # Random sample with replacement
                sample = relevant_column.sample().values[0]
                sample_loss += sample["loss"]
                sample_params += sample["params"]
                sample_actions += sample["m_actions"]
            sample_BIC = 2 * sample_loss + sample_params * np.log(sample_actions)
            sample_AIC = 2 * sample_loss + sample_params * 2
            bootstrapped_BICs_conds[condition][model].append(sample_BIC)
            bootstrapped_AICs_conds[condition][model].append(sample_AIC)

        

In [ ]:
# Calculate 95% CIs for each model for conditions separately

model_BIC_CIs_conds = {}
model_AIC_CIs_conds = {}

for condition in conditions:
    model_BIC_CIs_conds[condition] = {}
    model_AIC_CIs_conds[condition] = {}
    for model in models_to_bootstrap:
        model_BIC_CIs_conds[condition][model] = {}
        model_AIC_CIs_conds[condition][model] = {}
        bmean = np.mean(bootstrapped_BICs_conds[condition][model])
        bstd = np.std(bootstrapped_BICs_conds[condition][model])
        amean = np.mean(bootstrapped_AICs_conds[condition][model])
        astd = np.std(bootstrapped_AICs_conds[condition][model])

        model_BIC_CIs_conds[condition][model] = (bmean - 2 * bstd, bmean + 2 * bstd)
        model_AIC_CIs_conds[condition][model] = (amean - 2 * astd, amean + 2 * astd)

In [ ]:
# Plotting histograms of BICs
models_to_plot = [
    "2.0.3.0.0",
    "3.0.1.2",
    "2.0.1.1",
    #"3.1.1.1"
]

for cidx, condition in enumerate(conditions):
    min_BIC = float("inf")
    max_BIC = float("-inf")

    min_AIC = float("inf")
    max_AIC = float("-inf")

    for model in models_to_plot:
        model_min_BIC = min(bootstrapped_BICs_conds[condition][model])
        model_min_AIC = min(bootstrapped_AICs_conds[condition][model])
        model_max_BIC = max(bootstrapped_BICs_conds[condition][model])
        model_max_AIC = max(bootstrapped_AICs_conds[condition][model])

        if model_min_BIC < min_BIC:
            min_BIC = model_min_BIC

        if model_min_AIC < min_AIC:
            min_AIC = model_min_AIC

        if model_max_BIC > max_BIC:
            max_BIC = model_max_BIC

        if model_max_AIC > max_AIC:
            max_AIC = model_max_AIC

    BIC_bins = np.linspace(min_BIC - 1000, max_BIC + 1000, 100)
    AIC_bins = np.linspace(min_AIC - 1000, max_AIC + 1000, 100)

    colors = ['orange', 'blue', 'green', 'yellow']

    for idx, model in enumerate(models_to_plot):
        plt.figure((cidx)*2 + 1)
        plt.hist(bootstrapped_BICs_conds[condition][model], BIC_bins, alpha=0.5, label=model, color=colors[idx])
        plt.axvline(model_BIC_CIs_conds[condition][model][1], linestyle='--', color=colors[idx])
        plt.axvline(model_BIC_CIs_conds[condition][model][0], linestyle='-.', color=colors[idx])
        plt.title(f"{condition}_BIC")
        plt.legend()
        plt.figure((cidx)*2 + 2)
        plt.hist(bootstrapped_AICs_conds[condition][model], AIC_bins, alpha=0.5, label=model, color=colors[idx])
        plt.axvline(model_AIC_CIs_conds[condition][model][1], linestyle='--', color=colors[idx])
        plt.axvline(model_AIC_CIs_conds[condition][model][0], linestyle='-.', color=colors[idx])
        plt.title(f"{condition}_AIC")
        plt.legend()

plt.show()

In [ ]:
# Check overlap between confidence intervals
overlap_matrix = np.zeros((len(models_to_bootstrap), len(models_to_bootstrap)))

for i in range(len(overlap_matrix)):
    for j in range(i+1, len(overlap_matrix)):
        CI_i = model_BIC_CIs[models_to_bootstrap[i]]
        CI_j = model_BIC_CIs[models_to_bootstrap[j]]
        #print(CI_i, CI_j)
        if CI_i[0] <= CI_j[1] and CI_j[0] <= CI_i[1]:
            # Overlap in confidence intervals
            overlap_matrix[i][j] = min([CI_i[1]-CI_j[0], CI_j[1]-CI_i[0]])
            
overlap_matrix


In [ ]:
#  Inspecting the signal weight parameters for the models that have 3.1.x or 3.2.x

signal_wts = {}

for condition in conditions:
    signal_wts[condition] = {}
    for model in all_models:
        if not (".3.1." in model or ".3.2." in model):
            continue
        print(condition, model)
        signal_wts[condition][model] = []
        docs = collection.find({ "condition": condition, "model": model })
        for doc in docs:
            best_params = json.loads(doc["best_params"])
            signal_wts[condition][model].append(best_params["feedback_weight"])

In [ ]:
# Plot distributions of feedback_weight parameter

condition = "scarce"
model = "2.0.3.2.0"

data = np.array(signal_wts[condition][model])
mean = data.mean()
sd = data.std()

plt.hist(data)
plt.axvline(mean, linestyle='--', color='k')
plt.xlabel('Feedback weight parameter')
plt.ylabel('Frequency')
plt.title("{0} {1}, Mean: {2:0.2f}, SD: {3:0.2f}".format(condition, updated_model_name_map[model], mean,sd))
plt.show()

# 1.8 Creating Dataframes of Model Simulated Data

In [ ]:
# Creating dataframes of model simulated data for statistical analysis:
condition = "scarce"
models = {
    "control": "2.0.3.0.0",
    "scarce": "2.0.3.0.0"
}
control_model = models["control"]
scarce_model = models["scarce"]



In [ ]:
# Defining paths

# CM inferred strategies
results_path = '../results'
inferred_path = results_path + "/cm/inferred_strategies"
model_sim_path = results_path + "/model_sim_data"

scarce_inferred = inferred_path + f"/scarcity_scarce_{scarce_model}/strategies.pkl"
control_inferred = inferred_path + f"/scarcity_control_{control_model}/strategies.pkl"

# Expected scores of all strategies
strategy_scores_scarce_path = results_path + "/cm/strategy_scores/scarcity_scarce_clickcost_0.25_strategy_scores.pkl"
strategy_scores_control_path = results_path + "/cm/strategy_scores/scarcity_control_clickcost_1.0_strategy_scores.pkl"

model_data_df_rows = []


In [ ]:
# Reading inferred strategies

with open(scarce_inferred, 'rb') as f:
    scarce_strategies = pickle.load(f)
    
with open(control_inferred, 'rb') as f:
    control_strategies = pickle.load(f)

In [ ]:
# Reading strategy scores

with open(strategy_scores_scarce_path, 'rb') as file:
    strategy_scores_scarce = pickle.load(file)
    
with open(strategy_scores_control_path, 'rb') as file:
    strategy_scores_control = pickle.load(file)


In [ ]:
# Creating the strategy score dataframes

strategy_scores_scarce_list = [[k,v] for k,v in strategy_scores_scarce.items()]
strategy_scores_control_list = [[k,v] for k,v in strategy_scores_control.items()]

scarce_cluster_df = pd.DataFrame(strategy_scores_scarce_list, columns=["strategy", "score"])
control_cluster_df = pd.DataFrame(strategy_scores_control_list, columns=["strategy", "score"])

# Scale to 0-1:
scarce_cluster_df['scoreScaled'] = (scarce_cluster_df['score'] - scarce_cluster_df['score'].min()) / (scarce_cluster_df['score'].max() - scarce_cluster_df['score'].min())
control_cluster_df['scoreScaled'] = (control_cluster_df['score'] - control_cluster_df['score'].min()) / (control_cluster_df['score'].max() - control_cluster_df['score'].min())



In [ ]:
# Cluster the strategy scores

scarce_clusters = scipy.cluster.vq.kmeans(scarce_cluster_df['scoreScaled'], k_or_guess=[0.6,0.5,0.4])
scarce_cluster_centers = scarce_clusters[0]

control_clusters = scipy.cluster.vq.kmeans(control_cluster_df['scoreScaled'], k_or_guess=[0.6,0.5,0.4])
control_cluster_centers = control_clusters[0]

In [ ]:
# Compute the cluster labels: 0 - adaptive, 1 - moderately adaptive, 2 - maladaptive

scarce_clusters_repeated = np.repeat(np.expand_dims(scarce_cluster_centers, axis=0), len(scarce_cluster_df), axis=0)
scarce_cluster_distances = (scarce_clusters_repeated - np.repeat(np.expand_dims(np.array(scarce_cluster_df['scoreScaled']), axis=1),repeats=len(scarce_cluster_centers), axis=1))**2
scarce_cluster_labels = np.argmin(scarce_cluster_distances, axis=1)

control_clusters_repeated = np.repeat(np.expand_dims(control_cluster_centers, axis=0), len(control_cluster_df), axis=0)
control_cluster_distances = (control_clusters_repeated - np.repeat(np.expand_dims(np.array(control_cluster_df['scoreScaled']), axis=1),repeats=len(control_cluster_centers), axis=1))**2
control_cluster_labels = np.argmin(control_cluster_distances, axis=1)

In [ ]:
# Create dictionary of cluster labels:

scarce_cluster_dict = { strat: cluster for (strat, cluster) in zip(scarce_cluster_df.strategy, scarce_cluster_labels)}

control_cluster_dict = { strat: cluster for (strat, cluster) in zip(control_cluster_df.strategy, control_cluster_labels)}


In [ ]:

# Creating the trial dataframe

inf_strat = {
    "control": control_strategies,
    "scarce": scarce_strategies
}

strat_clust = {
    "control": control_cluster_dict,
    "scarce": scarce_cluster_dict
}

strat_scores = {
    "control": strategy_scores_control,
    "scarce": strategy_scores_scarce
}

scarcity_level = {
    "control": 1.0,
    "scarce": 0.25
}

trials_dicts = []

for condition in models.keys():
    docs = collection.find( { "condition": condition, "model": models[condition] } )
    for doc in docs:
        pid = doc["pid"]
        rewards = json.loads(doc['r'])
        expected = json.loads(doc['mer'])
        clicks = json.loads(doc['num_clicks'])

        part_data = all_trials_df.loc[all_trials_df.workerId == pid]
        num_simulations = 1 # range(len(rewards)) # Only doing for one simulation initially
        print(pid, len(part_data), num_simulations, models[condition])
        for sim_num in range(num_simulations): 
            new_pid = pid + str(sim_num)
            sim_r = rewards[sim_num]
            sim_mer = expected[sim_num]
            sim_clicks = clicks[sim_num]

            adjusted_sim_mer = [mer - c for (mer,c) in zip(sim_mer, sim_clicks)]
            for trial in range(len(sim_r)):
                strategy = inf_strat[condition][pid][trial]
                cluster = strat_clust[condition][strategy-1]
                strat_score = strat_scores[condition][strategy-1] / scarcity_level[condition]
                #print(trial, strategy, cluster,strat_score)
                trials_dicts.append({
                    "workerId": new_pid,
                    "trialNumbers": trial + 1,
                    "scarce": int(condition == "scarce"),
                    "trialScores": sim_r[trial],
                    "expectedScores": adjusted_sim_mer[trial],
                    "numClicks": sim_clicks[trial],
                    "numRewardedTrials": list(part_data.loc[part_data.trialNumbers == (trial + 1)].numRewardedTrials)[0],
                    "numUnrewardedTrials": list(part_data.loc[part_data.trialNumbers == (trial + 1)].numUnrewardedTrials)[0],
                    "strategy": strategy,
                    "avgClickLevel": list(part_data.loc[part_data.trialNumbers == (trial + 1)].avgClickLevel)[0],
                    "strategyScores": strat_score,
                    "rewardsWithheld": list(part_data.loc[part_data.trialNumbers == (trial + 1)].rewardsWithheld)[0],
                    "cluster": cluster
                })

model_df = pd.DataFrame(trials_dicts)
print("Done")

In [ ]:
save_folder = '../results/model_sim_data'
file_name = f"/c{control_model}_s{scarce_model}_{num_simulations}.csv"

model_df.to_csv(save_folder + file_name,index=False)

# 1.9 Comparing strategy evolution of models vs. participants

In [ ]:
# Creating dataframes of model simulated data for statistical analysis:

models = {
    "control": "2.0.3.3.0",
    "scarce": "2.0.3.0.0"
}
control_model = models["control"]
scarce_model = models["scarce"]

file_name = f"../results/model_sim_data/c{control_model}_s{scarce_model}_1.csv"

model_df = pd.read_csv(file_name)

model_dfs = {
    "control": model_df.loc[model_df.scarce == 0],
    "scarce": model_df.loc[model_df.scarce == 1]
}

part_dfs = {
    "control": all_trials_df.loc[all_trials_df.scarce == 0],
    "scarce": all_trials_df.loc[all_trials_df.scarce == 1]
}

In [ ]:
test = part_dfs["control"]

test.loc[test.strategy == 30]

In [ ]:
num_partitions = 5
num_top_strats = 6
num_end_strats = 3

begin_trials_until = {
    "control": 10,
    "scarce": 10,
} 
end_trials_from = {
    "control": 20,
    "scarce": 110,
} 

colors = ['red', 'blue','green','orange','black','purple','grey','brown','cyan','gold','crimson']

bar_width = 0.05
fig, ax = plt.subplots(2,2, figsize=(20,20))

for cond_id, condition in enumerate(models.keys()):
    part_df = part_dfs[condition]
    model_df = model_dfs[condition]
    partition_width = condition_trial_nums[condition] / num_partitions
#     begin_trials = df.loc[df.trialNumbers <= begin_trials_until[condition]]
#     strategy_counts = begin_trials["strategy"].value_counts()[0:num_begin_strats]
#     top_strategies = list(strategy_counts.index)
    
#     relevant_strats = top_strategies
    
#     end_trials = df.loc[df.trialNumbers >= end_trials_from[condition]]
#     strategy_counts = end_trials["strategy"].value_counts()[0:num_end_strats]
    part_strategy_counts = part_df["strategy"].value_counts()[0:num_top_strats]
    part_top_strategies = list(part_strategy_counts.index)
    print(part_strategy_counts)
    
    model_strategy_counts = model_df["strategy"].value_counts()[0:num_top_strats]
    model_top_strategies = list(model_strategy_counts.index)
    #print(model_strategy_counts)
    
    top_strategies = np.union1d(part_top_strategies, model_top_strategies)
    
#     print(part_top_strategies)
#     print(model_top_strategies)
    
    ind = np.arange(num_partitions)
    
    
    for df_id, df in enumerate([part_dfs[condition], model_dfs[condition]]):
        current_ax = ax[cond_id, df_id]
        for i, idx in enumerate(ind):
            relevant_data = df.loc[(df.trialNumbers > i*partition_width) & (df.trialNumbers <= (i+1) * partition_width)]
            for j, strat in enumerate(top_strategies):
                #print(len(relevant_data.loc[relevant_data.strategy.isin(top_strategies)])/len(relevant_data))
                strat_prop = len(relevant_data.loc[relevant_data.strategy == strat]) / len(relevant_data)
                current_ax.bar(idx + j*bar_width, strat_prop, bar_width, color=colors[j], label=strat if i==0 else None)
        current_ax.legend()
    
    #print(top_strategies)

plt.show()

In [ ]:
for strat in [21, 30, 31, 27, 22, 26, 24, 14, 76, 18, 2, 39, 3]:
    print(strat, control_cluster_dict[strat-1])


# Appendix

In [ ]:
# Checking control models that are missing

control_done_parts = control_part_BIC[~control_part_BIC['2.0.3.3.0'].isna()]
control_remaining_parts = set()
control_remaining_models = []
missing_models = 0
model_missing_parts_dict = {}
for col in control_done_parts.columns:
    model_missing_parts = list(control_done_parts.loc[control_done_parts[col].isna()].index)
    print("\n" + col)
    print("\n".join(model_missing_parts))
    if len(model_missing_parts) > 0:
        control_remaining_models.append(col)
        control_remaining_parts = control_remaining_parts.union(set(model_missing_parts))
        missing_models += len(model_missing_parts)
        model_missing_parts_dict[col] = model_missing_parts

In [ ]:
# Saving PIDS of missing models to files
for model, parts in model_missing_parts_dict.items():
    filename = f"control_remaining/{model}.txt"
    with open(filename, 'w') as f:
        f.write("\n".join(parts))

In [ ]:
# Checking scarce models that are missing

scarce_done_parts = scarce_part_BIC[~scarce_part_BIC['2.0.3.1.0'].isna()]
scarce_remaining_parts = set()
scarce_remaining_models = []
missing_models = 0
model_missing_parts_dict = {}
for col in scarce_done_parts.columns:
    model_missing_parts = list(scarce_done_parts.loc[scarce_done_parts[col].isna()].index)
    print("\n" + col)
    print("\n".join(model_missing_parts))
    if len(model_missing_parts) > 0:
        scarce_remaining_models.append(col)
        scarce_remaining_parts = scarce_remaining_parts.union(set(model_missing_parts))
        missing_models += len(model_missing_parts)
        model_missing_parts_dict[col] = model_missing_parts

In [ ]:
# Saving PIDS of missing models to files
for model, parts in model_missing_parts_dict.items():
    filename = f"scarce_remaining/{model}.txt"
    with open(filename, 'w') as f:
        f.write("\n".join(parts))

## Analyzing Inferred Strategies

In [ ]:
# Defining paths

# CM inferred strategies
results_path = '../results'
inferred_path = results_path + "/cm/inferred_strategies"

scarce_inferred = inferred_path + "/scarcity_scarce/strategies.pkl"
control_inferred = inferred_path + "/scarcity_control/strategies.pkl"

# Output files for strategies
scarce_file = results_path + "/mouselab-mdp-final-scarce.csv"
control_file = results_path + "/mouselab-mdp-final-control.csv"

# Expected scores of all strategies
strategy_scores_scarce_path = results_path + "/cm/strategy_scores/scarcity_scarce_clickcost_0.25_strategy_scores.pkl"
strategy_scores_control_path = results_path + "/cm/strategy_scores/scarcity_control_clickcost_1.0_strategy_scores.pkl"


scarce_df_rows = []
control_df_rows = []

In [ ]:
# Reading csv files

with open(scarce_file, newline='') as file:
    reader = csv.reader(file)
    for idx, row in enumerate(reader):
        if idx == 0: continue
        scarce_df_rows.append([row[0], int(row[1])])
        
with open(control_file, newline='') as file:
    reader = csv.reader(file)
    for idx, row in enumerate(reader):
        if idx == 0: continue
        control_df_rows.append([row[0], int(row[1])])

In [ ]:
# Reading inferred strategies

with open(scarce_inferred, 'rb') as f:
    scarce_strategies = pickle.load(f)
    
with open(control_inferred, 'rb') as f:
    control_strategies = pickle.load(f)

In [ ]:
strategy_df = all_trials_df[['workerId', 'trialNumbers', 'scarce']].copy()
strategy_df['strategy'] = [None] * len(strategy_df) 

In [ ]:
for workerId, stratList in scarce_strategies.items():
    try:
        strategy_df.loc[strategy_df.workerId == workerId, 'strategy'] = stratList
    except:
        continue
    
for workerId, stratList in control_strategies.items():
    try:
        strategy_df.loc[strategy_df.workerId == workerId, 'strategy'] = stratList
    except:
        continue

In [ ]:
# Reading strategy scores

with open(strategy_scores_scarce_path, 'rb') as file:
    strategy_scores_scarce = pickle.load(file)
    
with open(strategy_scores_control_path, 'rb') as file:
    strategy_scores_control = pickle.load(file)


In [ ]:
# Creating the strategy score dataframes

strategy_scores_scarce_list = [[k,v] for k,v in strategy_scores_scarce.items()]
strategy_scores_control_list = [[k,v] for k,v in strategy_scores_control.items()]

scarce_cluster_df = pd.DataFrame(strategy_scores_scarce_list, columns=["strategy", "score"])
control_cluster_df = pd.DataFrame(strategy_scores_control_list, columns=["strategy", "score"])

# Scale to 0-1:
scarce_cluster_df['scoreScaled'] = (scarce_cluster_df['score'] - scarce_cluster_df['score'].min()) / (scarce_cluster_df['score'].max() - scarce_cluster_df['score'].min())
control_cluster_df['scoreScaled'] = (control_cluster_df['score'] - control_cluster_df['score'].min()) / (control_cluster_df['score'].max() - control_cluster_df['score'].min())



In [ ]:
# Cluster the strategy scores

scarce_clusters = scipy.cluster.vq.kmeans(scarce_cluster_df['scoreScaled'], k_or_guess=[0.6,0.5,0.4])
scarce_cluster_centers = scarce_clusters[0]

control_clusters = scipy.cluster.vq.kmeans(control_cluster_df['scoreScaled'], k_or_guess=[0.6,0.5,0.4])
control_cluster_centers = control_clusters[0]

In [ ]:
# Compute the cluster labels: 0 - adaptive, 1 - moderately adaptive, 2 - maladaptive

scarce_clusters_repeated = np.repeat(np.expand_dims(scarce_cluster_centers, axis=0), len(scarce_cluster_df), axis=0)
scarce_cluster_distances = (scarce_clusters_repeated - np.repeat(np.expand_dims(np.array(scarce_cluster_df['scoreScaled']), axis=1),repeats=len(scarce_cluster_centers), axis=1))**2
scarce_cluster_labels = np.argmin(scarce_cluster_distances, axis=1)

control_clusters_repeated = np.repeat(np.expand_dims(control_cluster_centers, axis=0), len(control_cluster_df), axis=0)
control_cluster_distances = (control_clusters_repeated - np.repeat(np.expand_dims(np.array(control_cluster_df['scoreScaled']), axis=1),repeats=len(control_cluster_centers), axis=1))**2
control_cluster_labels = np.argmin(control_cluster_distances, axis=1)

In [ ]:
# Create dictionary of cluster labels:

scarce_cluster_dict = { strat: cluster for (strat, cluster) in zip(scarce_cluster_df.strategy, scarce_cluster_labels)}

control_cluster_dict = { strat: cluster for (strat, cluster) in zip(control_cluster_df.strategy, control_cluster_labels)}


In [ ]:
# Add clusters and strategy scores to strategy_df 

strategy_df['cluster'] = [None] * len(strategy_df)
strategy_df['strategyScores'] = [None] * len(strategy_df)

for idx, row in strategy_df.iterrows():
    if row['scarce'] == 0:
        strategy_df.at[idx, 'cluster'] = scarce_cluster_dict[row['strategy']-1]
        strategy_df.at[idx, 'strategyScores'] = strategy_scores_scarce[row['strategy']-1] / scarcity_level
    else:
        strategy_df.at[idx, 'cluster'] = control_cluster_dict[row['strategy']-1]
        strategy_df.at[idx, 'strategyScores'] = strategy_scores_control[row['strategy']-1]
    
    
strategy_df['strategyScores'] = strategy_df['strategyScores'].astype("float")

In [ ]:
# Merging trial data with strategy data

strategy_cols = ["workerId", "trialNumbers", "strategy", "cluster", "strategyScores"]

all_trials_df = pd.merge(all_trials_df, strategy_df[strategy_cols], on=['workerId', "trialNumbers"])


In [ ]:
all_trials_df

## Removing Outliers

In [ ]:
# Group statistics by participant
all_part_groups = all_trials_df.groupby('workerId')
control_part_groups = all_trials_df.loc[all_trials_df.scarce==0].groupby("workerId")
scarce_part_groups = all_trials_df.loc[all_trials_df.scarce==1].groupby("workerId")

# Get the averages of each statistic by participant
control_part_averages = control_part_groups.mean()
scarce_part_averages = scarce_part_groups.mean()
all_part_averages = all_part_groups.mean()

# Compute the proportion of trials with no clicks
all_part_averages['pctgSomeClick'] = (all_part_groups.apply(lambda x: x[x > 0].count()) / all_part_groups.count())['numClicks']
scarce_part_averages['pctgSomeClick'] = (scarce_part_groups.apply(lambda x: x[x > 0].count()) / scarce_part_groups.count())['numClicks']
control_part_averages['pctgSomeClick'] = (control_part_groups.apply(lambda x: x[x > 0].count()) / control_part_groups.count())['numClicks']

### Removing Based on Selected Criteria

In [ ]:
selected_exclusion_col = 'pctgSomeClick'

In [ ]:
# Removing outliers 

# Excluding those that have 100% of their trials with no clicks


zscore_threshold = (0.0 - all_part_averages[selected_exclusion_col].mean()) / all_part_averages[selected_exclusion_col].std()
excluded_parts_control = control_part_averages[control_part_averages[selected_exclusion_col] == 0]
included_parts_control = control_part_averages[control_part_averages[selected_exclusion_col] > 0]

zscore_threshold = (0.0 - scarce_part_averages[selected_exclusion_col].mean()) / scarce_part_averages[selected_exclusion_col].std()
excluded_parts_scarce = scarce_part_averages[scarce_part_averages[selected_exclusion_col] == 0]
included_parts_scarce = scarce_part_averages[scarce_part_averages[selected_exclusion_col] > 0]

num_excluded_parts = len(excluded_parts_scarce) + len(excluded_parts_control)

print("Number of participants excluded: {0} ({1:0.2f}%)".format(
    num_excluded_parts,
    100 * num_excluded_parts /len(all_trials_df.groupby("workerId"))
))
print("\tControl: {0} ({1:0.2f}%)".format(
    len(excluded_parts_control),
    100 * len(excluded_parts_control) / len(control_part_averages)
))
print("\tScarce: {0} ({1:0.2f}%)".format(
    len(excluded_parts_scarce),
    100 * len(excluded_parts_scarce) / len(scarce_part_averages)
))
included_part_IDs = list(included_parts_scarce.index) + list(included_parts_control.index)
outliers_excluded_part = all_trials_df.loc[all_trials_df.workerId.isin(included_part_IDs)].reset_index(drop=True)
outliers_excluded_part['trialNumbers'] = outliers_excluded_part['trialNumbers'].astype('int64')
outliers_excluded_part['scarce'] = outliers_excluded_part['scarce'].astype('int64')
outliers_excluded_part['numRewardedTrials'] = outliers_excluded_part['numRewardedTrials'].astype('int64')
outliers_excluded_part['numUnrewardedTrials'] = outliers_excluded_part['numUnrewardedTrials'].astype('int64')


print("Number of total trials excluded: {0} ({1:0.2f}%)".format(len(all_trials_df)-len(outliers_excluded_part), 100-100*len(outliers_excluded_part)/(len(all_trials_df))))
